In [744]:
import pandas as pd
import os, re
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine, types
from sqlalchemy import text 
from openpyxl import load_workbook

In [745]:
from dotenv import dotenv_values

config = dotenv_values()

# define variables for the login
pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [746]:
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [747]:
my_schema = 'team_5' # update it to your schema

with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))

In [748]:
df_1= pd.read_csv('../data/Raw/World_Marriage_Dataset.csv')

In [749]:
df_1.drop(columns=["Sr.No."], inplace=True)

In [750]:
df_1.columns = df_1.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [751]:
df_1.rename(columns={
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "dataprocess": "data_process",
    "Data Collection (Start Year)": "data_collection_start_year",
    "Data Collection (End Year)": "data_collection_end_year",
    "Data Source": "data_source",
    "Country": "country",
    "Sex": "sex"
}, inplace=True)

In [752]:
df_1.drop_duplicates(inplace=True)

df_1['data_collection_start_year'] = df_1['data_collection_start_year'].astype(str).str.replace(',', '').astype(int)
df_1['data_collection_end_year'] = df_1['data_collection_end_year'].astype(str).str.replace(',', '').astype(int)

In [753]:
df_1.isnull().sum()

country                       0
age_group                     0
sex                           0
marital_status                0
data_process                  0
data_collection_start_year    0
data_collection_end_year      0
data_source                   0
dtype: int64

In [754]:
#df_1.to_csv("cleaned_world_marriage.csv", index=False)

In [755]:
#df_1.to_sql('world_marriage', engine, if_exists='replace', index=False)

In [756]:
df_2 = pd.read_csv('../data/Raw/age-at-marriage-women.csv')

In [757]:
df_2.columns = df_2.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [758]:
df_2 = df_2.drop(columns=['1005564annotations'])

df_2.rename(columns={
    "entity": "country",   
}, inplace=True)

In [759]:
df_2.drop_duplicates(inplace=True)


In [760]:
df_2['year'] = df_2['year'].astype(str).str.replace(',', '').astype(int)

In [761]:
df_2.isnull().sum()

country                                0
code                                   0
year                                   0
mean_age_of_women_at_first_marriage    0
dtype: int64

In [762]:
#df_2.to_csv("cleaned_age_at_marriage_women.csv", index=False)

In [763]:
#df_2.to_sql('age_at_marriage_women', engine, if_exists='replace', index=False)

In [764]:
df_3= pd.read_csv('../data/Raw/marriage-rate-per-1000-inhabitants.csv')

In [765]:
df_3.columns = df_3.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [766]:
df_3.rename(columns={
    "entity": "country",   
}, inplace=True)

In [767]:
df_3['year'] = df_3['year'].astype(str).str.replace(',', '').astype(int)

In [768]:
df_3.drop_duplicates(inplace=True)


In [769]:
df_3.isnull().sum()

country                                          0
code                                             0
year                                             0
crude_marriage_rate_marriages_per_1000_people    0
dtype: int64

In [770]:
#df_3.to_csv("cleaned_marriage-rate-per-1000-inhabitants.csv", index=False)

In [771]:
#df_3.to_sql('married_rate_per_1000', engine, if_exists='replace', index=False)

In [772]:
df_4= pd.read_csv('../data/Raw/marriage-rates-in-1990-vs-2020.csv')

In [773]:
df_4.columns = df_4.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [774]:
df_4 = df_4.drop(columns=['worldregionsaccordingtoowid'])

df_4.rename(columns={
    "crudemarriageratemarriagesper1000people": "crude_marriage_rate",
    "crudemarriageratemarriagesper1000people1": "crude_marriage_rate_people1",
    "year1": "year_1",
    "entity": "country"
}, inplace=True)

In [775]:
df_4.drop_duplicates(inplace=True)
df_4.dropna(inplace=True)

In [776]:
df_4['year_1'] = pd.to_numeric(df_4['year_1'], errors='coerce').astype('Int64')

In [777]:
df_4.isnull().sum()

country                        0
code                           0
year                           0
crude_marriage_rate            0
crude_marriage_rate_people1    0
year_1                         0
dtype: int64

In [778]:
#df_4.to_csv("cleaned_marriage-rates-in-1990-vs-2020.csv", index=False)

In [779]:
#df_4.to_sql('marriage_rates_in_1990_vs_2020', engine, if_exists='replace', index=False)

In [780]:
df_5 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [781]:
df_5.columns = df_5.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [782]:

df_5.rename(columns={
    "shareofbirthsoutsideofmarriageofallbirths": "share_of_births_outside_of_marriage",
    "entity": "country"
}, inplace=True)

df_5.drop_duplicates(inplace=True)

In [783]:
df_5.isnull().sum()

country                                0
code                                   0
year                                   0
share_of_births_outside_of_marriage    0
dtype: int64

In [784]:
#df_5.to_csv("cleaned_share-of-births-outside-marriage.csv", index=False)

In [785]:
#df_5.to_sql('share_of_births_outside_marriage', engine, if_exists='replace', index=False)

In [786]:
df_6 = pd.read_csv('../data/Raw/share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv')

In [787]:
df_6.columns = df_6.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

df_6.drop_duplicates(inplace=True)
df_6.sample(5)

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
62,Women,NaN,45,84.2,91.4,95.4,86.7,74.5,NaN,NaN,NaN
38,Women,NaN,21,14.6,26.1,42.2,31.5,12.7,4.8,1.7,0.6
18,Men,NaN,35,84.8,86.7,89.1,73.7,56.3,44.2,NaN,NaN
23,Men,NaN,40,89.6,89.9,91.1,78.0,64.0,53.9,NaN,NaN
5,Men,NaN,22,13.5,16.8,25.8,18.8,6.8,2.5,1.1,0.4


In [788]:
df_6 = df_6.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_6.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)

In [789]:
df_6.isnull().sum()

sex                 0
year                0
1900_birthcohort    0
1920_birthcohort    0
1940_birthcohort    0
1960_birthcohort    0
1970_birthcohort    0
dtype: int64

In [790]:
#df_6.to_csv("cleaned_share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [791]:
#df_6.to_sql('men_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [792]:
df_7 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [793]:
df_7.columns = df_7.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [794]:
df_7.rename(columns={
    "shareofsingleparenthouseholds": "share_of_single_parent_households",
    "entity": "country"
}, inplace=True)

df_7.drop_duplicates(inplace=True)
df_7.sample(5)

,country,code,year,shareofbirthsoutsideofmarriageofallbirths
1107,Latvia,LVA,1978,12.0
65,Austria,AUT,1979,16.5
1224,Luxembourg,LUX,1983,8.1
707,Germany,DEU,1986,16.1
2005,Switzerland,CHE,2020,27.7


In [795]:
df_7.isnull().sum()

country                                      0
code                                         0
year                                         0
shareofbirthsoutsideofmarriageofallbirths    0
dtype: int64

In [796]:
#df_7.to_csv("cleaned_share-of-single-parent-households.csv", index=False)

In [797]:
#df_7.to_sql('single_parent_households', engine, if_exists='replace', index=False)

In [798]:
df_8 = pd.read_csv('../data/Raw/share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv')

In [799]:
df_8.columns = df_8.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [800]:
df_8['code'] = df_8['code'].fillna('GBR')
df_8.sample(5)

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
33,Men,GBR,50,92.9,91.9,92.4,81.7,70.8,NaN,NaN,NaN
40,Women,GBR,23,32.7,49.5,68.2,48.4,24.0,10.0,4.7,1.6
17,Men,GBR,34,83.2,85.5,88.5,72.4,53.7,40.9,NaN,NaN
6,Men,GBR,23,21.4,26.8,38.1,26.2,10.5,3.9,2.1,0.8
47,Women,GBR,30,71.6,82.7,90.9,76.7,55.8,37.8,28.9,NaN


In [801]:
df_8 = df_8.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_8.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)

df_8.drop_duplicates(inplace=True)
df_8.sample(5)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
6,Men,23,21.4,26.8,38.1,26.2,10.5
54,Women,37,80.6,89.4,94.4,84.1,69.4
19,Men,36,86.1,87.6,89.7,74.8,58.3
14,Men,31,76.5,79.7,85.1,66.6,45.1
52,Women,35,79.0,88.3,93.8,82.8,66.7


In [802]:
df_8.isnull().sum()

sex                 0
year                0
1900_birthcohort    0
1920_birthcohort    0
1940_birthcohort    0
1960_birthcohort    0
1970_birthcohort    0
dtype: int64

In [803]:
#df_8.to_csv("cleaned_share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [804]:
#df_8.to_sql('women_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [805]:
#pip install openpyxl pywin32

In [806]:
df_excel_1 = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')

In [807]:
#all_sheets = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx', sheet_name=None)

In [808]:
xls_1 = pd.ExcelFile('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')
print(xls_1.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']


In [809]:
excel_1 = '../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx'

# Output directory (make sure it exists)
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

# List of sheets you want to extract
sheets_to_extract = ['MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']

In [810]:
"""for sheet in sheets_to_extract:
    # Read just this sheet into a DataFrame
    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)
    
    # Optional: Clean the filename (replace spaces with underscores, etc.)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    
    # Save the DataFrame as CSV
    df_excel_1.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")
"""

'for sheet in sheets_to_extract:\n    # Read just this sheet into a DataFrame\n    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)\n    \n    # Optional: Clean the filename (replace spaces with underscores, etc.)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    \n    # Save the DataFrame as CSV\n    df_excel_1.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n'

In [811]:
xls_2 = pd.ExcelFile('../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx')
print(xls_2.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'FERTILITY INDICATORS']


In [812]:
excel_2 = '../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx'
sheet_name = 'FERTILITY INDICATORS'
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

df_excel_2 = pd.read_excel(excel_2, sheet_name=sheet_name)


In [813]:
"""csv_name = sheet_name.replace(' ', '_').lower() + '.csv'
csv_path = os.path.join(output_dir, csv_name)
df_excel_2.to_csv(csv_path, index=False)
print(f"Saved: {csv_path}")
"""

'csv_name = sheet_name.replace(\' \', \'_\').lower() + \'.csv\'\ncsv_path = os.path.join(output_dir, csv_name)\ndf_excel_2.to_csv(csv_path, index=False)\nprint(f"Saved: {csv_path}")\n'

In [814]:
xls_3 = pd.ExcelFile('../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx')
print(xls_3.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'Countries', 'Regions']


In [815]:
excel_3 = '../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx'
sheets_to_extract = ['Countries', 'Regions']
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)


In [816]:
"""
for sheet in sheets_to_extract:
    df = pd.read_excel(excel_3, sheet_name=sheet)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    df.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")

"""

'\nfor sheet in sheets_to_extract:\n    df = pd.read_excel(excel_3, sheet_name=sheet)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    df.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n\n'

In [817]:
df_9 = pd.read_csv('../data/Raw/unpopulation_dataportal_20250728095844.csv')
df_9.sample(5)

,IndicatorId,IndicatorName,IndicatorShortName,Source,SourceYear,Author,LocationId,Location,Iso2,Iso3,...,AgeStart,AgeEnd,Age,CategoryId,Category,EstimateTypeId,EstimateType,EstimateMethodId,EstimateMethod,Value
1405,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,50,Bangladesh,BD,BGD,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,76.54
20128,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,694,Sierra Leone,SL,SLE,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,68.94
14930,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,508,Mozambique,MZ,MOZ,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,74.47
14881,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,504,Morocco,MA,MAR,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,57.99
10977,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,388,Jamaica,JM,JAM,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,56.24


In [818]:
df_9.columns = df_9.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_9.sample(5)

,indicatorid,indicatorname,indicatorshortname,source,sourceyear,author,locationid,location,iso2,iso3,...,agestart,ageend,age,categoryid,category,estimatetypeid,estimatetype,estimatemethodid,estimatemethod,value
19809,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,686,Senegal,SN,SEN,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,65.56
658,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,28,Antigua and Barbuda,AG,ATG,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,3,Projection,30.62
13127,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,446,"China, Macao SAR",MO,MAC,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,46.03
20037,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,690,Seychelles,SC,SYC,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,45.10
11757,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,410,Republic of Korea,KR,KOR,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,3,Projection,43.16


In [819]:
df_9 = df_9.drop(columns=['indicatorid','indicatorshortname',
    'source',
    'author', 'locationid', 'iso2','estimatetypeid','category','categoryid','agestart','ageend','author','ageid', 'estimatetype','variantid','sexid','timeid'])

df_9.rename(columns={
    "sourceyear": "year",
    "location": "country",
    "estimatemethodid": "estimate_method",
    "iso3": "code",
}, inplace=True)



In [820]:
df_9.drop_duplicates(inplace=True)

In [821]:
df_9

,indicatorname,year,country,code,time,variant,sex,age,estimate_method,estimatemethod,value
0,Currently married (Percent),2024,Afghanistan,AFG,1970,Median,Female,15-49,2,Interpolation,80.94
2,Currently married (Percent),2024,Afghanistan,AFG,1971,Median,Female,15-49,2,Interpolation,80.90
4,Currently married (Percent),2024,Afghanistan,AFG,1972,Median,Female,15-49,2,Interpolation,80.87
6,Currently married (Percent),2024,Afghanistan,AFG,1973,Median,Female,15-49,2,Interpolation,80.84
8,Currently married (Percent),2024,Afghanistan,AFG,1974,Median,Female,15-49,2,Interpolation,80.53
...,...,...,...,...,...,...,...,...,...,...,...
25078,Currently married (Percent),2024,Zambia,ZMB,2021,Median,Female,15-49,3,Projection,54.31
25080,Currently married (Percent),2024,Zambia,ZMB,2022,Median,Female,15-49,3,Projection,53.82
25082,Currently married (Percent),2024,Zambia,ZMB,2023,Median,Female,15-49,3,Projection,53.35
25084,Currently married (Percent),2024,Zambia,ZMB,2024,Median,Female,15-49,3,Projection,52.91


In [822]:
df_9.isnull().sum()

indicatorname      0
year               0
country            0
code               0
time               0
variant            0
sex                0
age                0
estimate_method    0
estimatemethod     0
value              0
dtype: int64

In [823]:
#df_9.to_csv("cleaned_unpopulation_dataportal.csv", index=False)

In [824]:
#df_9.to_sql('unpopulation_dataportal', engine, if_exists='replace', index=False)

In [825]:
df_10 = pd.read_csv('../data/processed/countries_un.csv',  header=5, low_memory=False)

In [826]:
df_10.columns = (
    df_10.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
)
df_10.sample(10)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,dataprocess
42285,Finland,246,Married or in-union women,1990,40-44,77.550000,165.839512,Estimate
80955,Malta,470,Married or in-union women,2045,30-34,64.621860,9.056754,Projection
117622,Singapore,702,Married or in-union women,2011,45-49,78.839130,143.599168,Estimate
22054,Cayman Islands,136,Married or in-union women,1972,45-49,80.770000,0.225348,Estimate
83668,Mongolia,496,Married or in-union women,1979,35-39,91.710000,34.840629,Estimate
127849,Tajikistan,762,Married or in-union women,1994,20-24,71.916098,191.327384,Estimate
12219,Bolivia (Plurinational State of),68,Married or in-union women,2039,30-34,58.512295,332.087702,Projection
26628,Colombia,170,Married or in-union women,1977,35-39,74.353750,430.426795,Estimate
11908,Bolivia (Plurinational State of),68,Married or in-union women,2000,35-39,83.090000,198.399394,Estimate
99807,Panama,591,Married or in-union women,1971,15-49,59.439294,204.164466,Estimate


In [827]:
df_10.rename(columns={
    "dataprocess": "data_process",
}, inplace=True)

df_10.drop_duplicates(inplace=True)
df_10.sample(5)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,data_process
28211,Mayotte,175,Married or in-union women,2013,30-34,70.616654,6.789085,Estimate
108162,Romania,642,Married or in-union women,2044,25-29,46.803192,218.746185,Projection
32560,Cyprus,196,Married or in-union women,1990,15-19,2.755064,0.817855,Estimate
39095,Eritrea,232,Married or in-union women,1996,15-49,66.332869,336.013458,Estimate
85369,Montenegro,499,Married or in-union women,2030,20-24,14.455283,2.646762,Projection


In [828]:
for col in ['percentage', 'number']:
    if col in df_10.columns:
        df_10[col] = (
            df_10[col]
            .astype(str)
            .str.replace(',', '.', regex=False)
            .str.extract(r'([-+]?[0-9]*\.?[0-9]+)', expand=False)
            .astype(float)
            .round(2)
        )

In [829]:
unnamed_cols = [col for col in df_10.columns if 'unnamed' in col.lower()]
df_10.drop(columns=unnamed_cols, inplace=True)

In [830]:
df_10.dropna(inplace=True)

In [831]:
df_10.isnull().sum()

countryorarea    0
isocode          0
indicator        0
year             0
agegroup         0
percentage       0
number           0
data_process     0
dtype: int64

In [832]:
df_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145800 entries, 0 to 145799
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   countryorarea  145800 non-null  object 
 1   isocode        145800 non-null  int64  
 2   indicator      145800 non-null  object 
 3   year           145800 non-null  int64  
 4   agegroup       145800 non-null  object 
 5   percentage     145800 non-null  float64
 6   number         145800 non-null  float64
 7   data_process   145800 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 8.9+ MB


In [833]:
#df_10.to_csv("cleaned_countries_1970_2025_un.csv", index=False)

In [834]:
#df_10.to_sql('countries_1970_2025_un', engine, if_exists='replace', index=False)

In [835]:
df_11 = pd.read_csv('../data/processed/currently_married_un.csv',  header=2, low_memory=False)

In [836]:
df_11.sample(8)

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
7776,Chile,152,1992,1992,Men,[40-44],40,44,82.88,Census,1992 Census,861,Chile 1992 Census,UNSD,1.0,NaN,NaN
40416,Rwanda,646,2005,2005,Women,[75+],75,999,18.00,Survey,2005 DHS,1717,Rwanda 2005 Demographic and Health Survey,DHS_HH,1.0,NaN,NaN
17293,Germany,276,2014,2014,Men,[65-69],65,69,79.11,Estimate,2014 Estimate,2102,Germany 2014 Estimate,UNSD,NaN,NaN,NaN
43237,Sierra Leone,694,1992,1992,Women,[35-39],35,39,86.70,Survey,1992 DSMS,4549,Sierra Leone 1992 Demographic and Social Situa...,National statistics,NaN,NaN,NaN
22241,Iceland,352,2018,2018,Men,[25-29],25,29,12.11,Estimate,2018 Estimate,2121,Iceland 2018 Estimate,UNSD,NaN,NaN,NaN
10891,Czechia,203,2012,2012,Men,[70-74],70,74,78.20,Estimate,2012 Estimate,2079,Czech Republic 2012 Estimate,UNSD,NaN,NaN,NaN
32861,Myanmar,104,2015,2016,Men,[25-29],25,29,60.94,Survey,2015-16 MDHS,6084,Myanmar 2015-2016 Demographic and Health Survey,DHS_HH,NaN,NaN,NaN
2787,Bangladesh,50,2004,2004,Women,[30-34],30,34,92.70,Survey,2004 DHS,1813,Bangladesh 2004 Demographic and Health Survey,DHS_STATcompiler,NaN,NaN,NaN


In [837]:
df_11.columns = (
    df_11.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_11.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
31431,Mexico,484,2009,2009,Men,[25-29],25,29,54.64,Survey,2009 ENADID,5550,Mexico 2009 Encuesta Nacional de la Dinámica D...,UNSD,1.0,NaN,NaN
19910,Hungary,348,1975,1975,Women,[30-34],30,34,86.64,Estimate,1975 Estimate,2120,Hungary 1975 Estimate,UNSD,NaN,NaN,NaN
34090,Netherlands,528,2007,2007,Women,[45-49],45,49,69.99,Estimate,2007 Estimate,2170,Netherlands 2007 Estimate,UNSD,NaN,NaN,NaN
15826,France,250,1982,1982,Men,[55-59],55,59,84.06,Census,1982 Census,363,France 1982 Census,UNSD,NaN,Based on a sample of census returns.,Excluding diplomatic personnel outside the cou...
2576,Bahrain,48,1995,1995,Women,[30-34],30,34,75.70,Survey,1995 FHS,181,Bahrain 1995 Family Health Survey,GFHS,NaN,NaN,NaN
19820,Hungary,348,1972,1972,Men,[70-74],70,74,77.31,Estimate,1972 Estimate,2120,Hungary 1972 Estimate,UNSD,NaN,NaN,NaN
47226,Sweden,752,2006,2006,Women,[70-74],70,74,52.36,Estimate,2006 Estimate,2227,Sweden 2006 Estimate,UNSD,NaN,NaN,NaN
51947,Uruguay,858,1985,1985,Men,[15-19],15,19,1.83,Census,1985 Census,241,Uruguay 1985 Census,UNSD,1.0,Data have not been adjusted for underenumeration.,NaN


In [838]:
df_11 = df_11.drop(columns = ['datacataloglongname', 'datacatalogid', 'yearstart' , 'yearend', 'noteondata', 'noteoncountryandpopulation', 'including_consensual_unions'])

df_11.rename(columns={
    "agestart": "age_start",
    "countryorarea": "country",
    "datasource": "data_source",
    "datavalue" : "data_value"
}, inplace=True)

df_11.sample(10)

,country,isocode,sex,agegroup,age_start,ageend,data_value,dataprocess,datacatalogshortname,data_source
50279,Turkey,792,Women,[10-14],10,14,0.00,Estimate,2013 Estimate,UNSD
13396,Estonia,233,Men,[50-54],50,54,73.00,Census,2000 Census,UNSD
9337,Costa Rica,188,Women,[20-24],20,24,49.43,Census,1984 Census,UNSD
13155,El Salvador,222,Women,[45-49],45,49,68.50,Survey,1985 DHS,DHS_STATcompiler
34364,Netherlands,528,Women,[35-39],35,39,52.15,Estimate,2018 Estimate,UNSD
48932,Timor-Leste,626,Men,[40-44],40,44,93.90,Survey,2009-2010 DHS,DHS_STATcompiler
32135,Montserrat,500,Women,[15-19],15,19,0.33,Census,1980 Census,UNSD
4220,Bosnia and Herzegovina,70,Women,[45-49],45,49,86.12,Survey,2011-2012 MICS,MICS
33243,Nepal,524,Women,[25-29],25,29,93.20,Survey,2001 DHS,DHS_STATcompiler
19428,Haiti,332,Women,[60-64],60,64,51.99,Census,1982 Census,UNSD


In [839]:
df_11.drop_duplicates(inplace=True)

In [840]:
df_11.isnull().sum()

country                 0
isocode                 0
sex                     0
agegroup                0
age_start               0
ageend                  0
data_value              0
dataprocess             0
datacatalogshortname    0
data_source             0
dtype: int64

In [841]:
#df_11.to_csv("cleaned_currently_married_un.csv", index=False)

In [842]:
#df_11.to_sql('currently_married_un', engine, if_exists='replace', index=False)

In [843]:
df_12 = pd.read_csv('../data/processed/ever_married_un.csv', header= 2, low_memory = False)
df_12.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
0,Afghanistan,4,1972,1974,Men,[15-19],15,19,7.7,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,[20-24],20,24,32.6,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,[25-29],25,29,61.4,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,[30-34],30,34,83.0,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,[35-39],35,39,91.2,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN


In [844]:
df_12.columns = (
    df_12.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_12.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
30920,Lithuania,440,2015,2015,Women,[10-14],10,14,0.00,Estimate,2015 Estimate,2146,Lithuania 2015 Estimate,UNSD,NaN,NaN,NaN
53502,United Kingdom,826,2010,2010,Women,[20-24],20,24,6.27,Estimate,2010 Estimate,2246,United Kingdom 2010 Estimate,UNSD,NaN,NaN,Data pertain to England and Wales only. | Excl...
20833,Haiti,332,2005,2006,Women,[55-59],55,59,99.50,Survey,2005-2006 DHS,1785,Haiti 2005-2006 Demographic and Health Survey,DHS_HH,NaN,NaN,NaN
17897,Germany,276,1987,1987,Women,[35-39],35,39,90.86,Census,1987 Census,5308,Germany 1987 Census,IPUMS,NaN,Data are based on a 5 per cent sample of censu...,Data pertain to West Germany.
51206,Togo,768,2010,2010,Men,[35-39],35,39,89.85,Census,2010 Census,5267,Togo 2010 Census,INED,1.0,NaN,NaN
31353,Madagascar,450,2000,2000,Women,[20-24],20,24,75.38,Survey,2000 MICS_HH,1891,Madagascar 2000 Multiple Indicator Cluster Survey,MICS_HH,NaN,NaN,NaN
13873,Egypt,818,2000,2000,Men,[40-44],40,44,97.90,Survey,2000 DHS,1830,Egypt 2000 Demographic and Health Survey,DHS_HH,NaN,NaN,NaN
28968,Latvia,428,2011,2011,Women,[45-49],45,49,88.52,Estimate,2011 Estimate,2142,Latvia 2011 Estimate,UNSD,NaN,NaN,NaN


In [845]:
df_12 = df_12.drop(columns = ['yearstart', 'yearend', 'datacatalogshortname', 'datacatalogid', 'datacataloglongname', 'including_consensual_unions', 'noteondata', 'noteoncountryandpopulation'])

df_12.rename(columns={
    "agestart": "age_start",
    "ageend": "age_end",
    "countryorarea": "country"
}, inplace=True)
df_12.sample(8)

,country,isocode,sex,agegroup,age_start,age_end,datavalue,dataprocess,datasource
40083,Philippines,608,Men,[65-69],65,69,97.19,Census,UNSD
9045,Colombia,170,Men,[50-54],50,54,90.70,Census,National statistics
14235,Equatorial Guinea,226,Women,[60-64],60,64,93.57,Survey,MICS_HH
31721,Malaysia,458,Women,[55-59],55,59,97.72,Census,UNSD
37578,Norway,578,Women,[65-69],65,69,83.87,Estimate,UNSD
27534,Kazakhstan,398,Women,[60-64],60,64,98.69,Census,National statistics
41580,Réunion,638,Women,[75+],75,999,82.34,Census,UNSD
39576,Paraguay,600,Women,[50-54],50,54,84.49,Census,UNSD


In [846]:
df_12.dropna(inplace=True)

In [847]:
df_12.isnull().sum()

country        0
isocode        0
sex            0
agegroup       0
age_start      0
age_end        0
datavalue      0
dataprocess    0
datasource     0
dtype: int64

In [848]:
#df_12.to_csv("cleaned_ever_married_un.csv", index=False)

In [849]:
#df_12.to_sql('ever_married_un', engine, if_exists= 'replace', index= False)

In [850]:
df_13 = pd.read_csv('../data/processed/fertility_indicators_un.csv', header=6, low_memory=False)
df_13.head()

,Country or Area,Country or Area Code,Age Group,Indicator,Date,Value,Series,DataType,Data Source Type,Survey Programme,Data Source Inventory ID,Data Source Name,Data Source Name (short),Data Source Start Year,Data Source End Year,Reference,Reference Year
0,Afghanistan,4,[Total],TFR,1964.977051,7.966653,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
1,Afghanistan,4,[Total],TFR,1965.977051,8.212275,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
2,Afghanistan,4,[Total],TFR,1966.977051,8.317603,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
3,Afghanistan,4,[Total],TFR,1967.977051,8.225812,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
4,Afghanistan,4,[Total],TFR,1968.977051,8.068459,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012


In [851]:
df_13.columns = (df_13.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '')
        .str.replace('(', '')
        .str.replace(')', '')
        .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
        )

df_13.sample(6)

,countryorarea,countryorareacode,agegroup,indicator,date,value,series,datatype,datasourcetype,surveyprogramme,datasourceinventoryid,datasourcename,datasourcenameshort,datasourcestartyear,datasourceendyear,reference,referenceyear
26401,French Guiana,254,[35-39],ASFR3539,2004.500000,89.660000,"Register,Direct,CAG-BilanDemog2005,450-16-39269",Direct,Register,VR,450,Vital Registration,Register,2004,2004,CAG-BilanDemog2005,2005
60055,Russian Federation,643,[Total],MAC,2003.500000,26.250000,NSO.20170421,Direct,Register,VR,545,Vital Registration,Register,2003,2003,Central Statistics Database,2017
23601,Estonia,233,[30-34],ASFR3034,1979.500000,55.160000,Eurostat.20190531,Official estimates,Estimate,Estimate,2089,All sources of estimates,Estimates,1979,1979,"Eurostat Statistics, Fertility rates by age [d...",2019
71097,Trinidad and Tobago,780,[Total],TFR,1967.500000,3.850256,"Register,Computed rate from DYB,DYB,567-135-22",Computed rate from DYB,Register,VR,567,Vital Registration,Register,1967,1967,Demographic Yearbook,1969
58314,Republic of Moldova,498,[20-24],ASFR2024,1988.500000,193.880000,NSO.201510,Direct,Register,VR,614,Vital Registration,Register,1988,1988,NSO website,2015
31896,Haiti,332,[20-24],ASFR2024,2004.620465,159.241900,"2016 DHS,Birth Histories,FBH analysis 2018,582...",Birth histories,Survey,DHS,5825,Haiti 2016 Demographic and Health Survey (DHS),2016 DHS,2016,2017,Fertility rates from full birth histories anal...,2018


In [852]:
df_13 = df_13.drop(columns=['countryorareacode','indicator','datasourceinventoryid','surveyprogramme','series','datasourcename','reference','referenceyear'])

df_13.replace({
    "agegroup": "age_group",
    "countryorarea": "country",
    "datatype": "data_type",
},inplace=True)

In [853]:
df_13['date'] = df_13['date'].astype(int)
df_13['value'] = df_13['value'].round(2)
df_13.sample(12)

,countryorarea,agegroup,date,value,datatype,datasourcetype,datasourcenameshort,datasourcestartyear,datasourceendyear
74647,United Republic of Tanzania,[30-34],2007,217.00,Direct,Survey,2010 DHS,2009,2010
66121,Sri Lanka,[35-39],1967,145.50,Computed rate from reported ASFR,Register,Register,1967,1967
57453,Puerto Rico,[40-44],1979,11.51,Computed rate from DYB,Register,Register,1979,1979
69293,TFYR Macedonia,[40-44],1954,73.15,Fertility data (adjusted),Estimate,Estimates,1954,1954
57215,Portugal,[45-49],2014,0.56,Official estimates,Estimate,Estimates,2014,2014
50516,Nepal,[15-19],1995,155.00,Official estimates,Estimate,Estimates,1995,1995
6671,Belarus,[25-29],1977,124.40,Fertility data (adjusted),Estimate,Estimates,1977,1977
398,Albania,[Total],1964,5.66,Fertility data (adjusted),Estimate,Estimates,1964,1964
35747,Iran (Islamic Republic of),[35-39],1996,56.60,Own-children method,Census,1996 Census,1996,1996
68235,Switzerland,[30-34],1997,96.34,Official estimates,Estimate,Estimates,1997,1997


In [854]:
#df_13.to_csv("cleaned_fertility_indicators.csv", index=False)

In [855]:
#df_13.to_sql('fertility_indicators_un',engine, if_exists='replace', index=False)

In [856]:
df_14 = pd.read_csv('../data/processed/marital_status_by_age_un.csv', header= 2, low_memory=False)
df_14.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,MaritalStatus,Non-standard_AgeGroups,Series_contains_Non-standard_AgeGroups,AgeGroup,AgeStart,...,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Age groups,Note on Marital Status,Note on Data,Note on Country and Population,Note Other
0,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[15-19],15,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[20-24],20,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[25-29],25,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[30-34],30,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[35-39],35,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN


In [857]:
df_14.columns= (df_14.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '' , regex=True)  
    )
df_14.sample(5)

,countryorarea,isocode,yearstart,yearend,sex,maritalstatus,nonstandard_agegroups,series_contains_nonstandard_agegroups,agegroup,agestart,...,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteonagegroups,noteonmaritalstatus,noteondata,noteoncountryandpopulation,noteother
226297,South Africa,710,1991,1991,Women,Single,NaN,NaN,[70-74],70,...,1991 Census,1360,South Africa 1991 Census,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
217419,Senegal,686,2017,2017,Men,Widowed,NaN,NaN,[35-39],35,...,2017 DHS,7630,Senegal 2017 Demographic and Health Survey,DHS_STATcompiler,NaN,NaN,NaN,NaN,NaN,NaN
83040,France,250,1985,1985,Men,Married,NaN,NaN,[40-44],40,...,1985 Estimate,2094,France 1985 Estimate,UNSD,NaN,NaN,NaN,NaN,Excluding diplomatic personnel outside the cou...,NaN
125476,Israel,376,2009,2009,Men,Married,NaN,NaN,[65-69],65,...,2009 Estimate,2127,Israel 2009 Estimate,UNSD,NaN,Totals may not add up to the sum of the respec...,NaN,NaN,Including data for East Jerusalem and Israeli ...,NaN
268850,Zambia,894,1996,1997,Women,Never married,NaN,NaN,[25-29],25,...,1996 DHS,1699,Zambia 1996 Demographic and Health Survey,DHS_STATcompiler,NaN,NaN,NaN,NaN,NaN,NaN


In [858]:
df_14 = df_14.drop(columns=['datacataloglongname', 'noteondata', 'noteoncountryandpopulation','noteonagegroups', 'noteother',
                             'including_consensual_unions','isocode', 'datacatalogid', 'noteonmaritalstatus', 'series_contains_nonstandard_agegroups','nonstandard_agegroups'])

df_14.rename(columns={
    "countryorarea": "country",
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "yearstart": "year_start",
    "yearend": "year_end",
    }, inplace =True
    )

df_14.sample(10)

,country,year_start,year_end,sex,marital_status,age_group,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datasource
22580,Brazil,1980,1980,Men,Married,[20-24],20,24,23.69,Census,1980 Census,UNSD
239449,Switzerland,1996,1996,Women,Divorced,[45-49],45,49,13.15,Estimate,1996 Estimate,UNSD
51232,Costa Rica,2015,2015,Women,Married,[25-29],25,29,23.15,Estimate,2015 Estimate,UNSD
239601,Switzerland,1998,1998,Women,Widowed,[35-39],35,39,0.77,Estimate,1998 Estimate,UNSD
67432,Ecuador,1974,1974,Women,Single,[20-24],20,24,40.71,Census,1974 Census,UNSD
144524,Liechtenstein,2000,2000,Men,Divorced,[60-64],60,64,5.42,Estimate,2000 Estimate,UNSD
74448,Faeroe Islands,2011,2011,Women,Married,[65-69],65,69,69.37,Census,2011 Census,National statistics
64013,Denmark,2013,2013,Women,Single,[45-49],45,49,19.07,Estimate,2013 Estimate,UNSD
245012,Togo,1970,1970,Women,Married or in consensual union,[30-34],30,34,93.04,Census,1970 Census,UNSD
109311,Hungary,2018,2018,Men,Divorced,[25-29],25,29,0.59,Estimate,2018 Estimate,UNSD


In [859]:
df_14.drop_duplicates(inplace=True)
df_14.isnull().sum()

country                 0
year_start              0
year_end                0
sex                     0
marital_status          0
age_group               0
agestart                0
ageend                  0
datavalue               0
dataprocess             0
datacatalogshortname    0
datasource              0
dtype: int64

In [860]:
#df_14.to_csv("cleaned_marital_status_by_age_un.csv", index=False)

In [861]:
#df_14.to_sql('marital_status_by_age_un', engine, if_exists='replace', index=False)

In [862]:
df_15 = pd.read_csv('../data/processed/regions_un.csv', header=5, low_memory= False)
df_15.head(10)

,Region and subregion,ISO code,Regional Classification,Indicator,Year,AgeGroup,Percentage,Number,DataProcess
0,World,900,M49,Married or in-union women,1970,15-19,22.576683,7.186782e+04,Estimate
1,World,900,M49,Married or in-union women,1970,20-24,63.802057,1.628604e+05,Estimate
2,World,900,M49,Married or in-union women,1970,25-29,87.174827,1.826811e+05,Estimate
3,World,900,M49,Married or in-union women,1970,30-34,90.825027,1.791214e+05,Estimate
4,World,900,M49,Married or in-union women,1970,35-39,90.284386,1.615263e+05,Estimate
5,World,900,M49,Married or in-union women,1970,40-44,86.483531,1.393344e+05,Estimate
6,World,900,M49,Married or in-union women,1970,45-49,82.680237,1.160884e+05,Estimate
7,World,900,M49,Married or in-union women,1970,15-49,69.379111,1.013480e+06,Estimate
8,World,900,M49,Married or in-union women,1971,15-19,22.630416,7.412762e+04,Estimate
9,World,900,M49,Married or in-union women,1971,20-24,63.613178,1.700873e+05,Estimate


In [863]:
df_15.columns = (df_15.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(','')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
    )
df_15.sample(6)

,regionandsubregion,isocode,regionalclassification,indicator,year,agegroup,percentage,number,dataprocess
26268,Lower-middle-income countries,1501,Income group,Married or in-union women,2013,35-39,89.421682,176789.388846,Estimate
9281,Central Asia,5500,SDG-M49,Married or in-union women,1996,20-24,60.025997,2874.747027,Estimate
10709,Southern Asia,5501,SDG-M49,Married or in-union women,2012,40-44,89.536677,90170.624488,Estimate
24117,Other developing countries,934,Development group,Married or in-union women,1987,40-44,88.235734,67267.379949,Estimate
4056,Europe and Northern America,513,SDG,Married or in-union women,1991,15-19,7.685652,2642.798423,Estimate
25913,Low-income countries,1500,Income group,Married or in-union women,2050,20-24,46.658067,58837.237221,Projection


In [864]:
df_15 = df_15.drop(columns=['regionalclassification'])

df_15.rename(columns={
    "regionandsubregion": "region",
    "isocode": "iso_code",
    "agegroup": "age_group",
    "dataprocess": "process"
}, inplace=True)

df_15.sample(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
24009,Other developing countries,934,Married or in-union women,1974,20-24,63.243094,73034.429237,Estimate
17502,South America,931,Married or in-union women,1970,45-49,74.367035,5850.725361,Estimate
24629,Least developed countries,941,Married or in-union women,1970,40-44,81.870404,5894.647772,Estimate
8861,Asia,935,Married or in-union women,2024,40-44,89.292037,283836.509608,Estimate
7486,Southern Africa,913,Married or in-union women,2014,45-49,57.225779,1921.072824,Estimate
4719,Africa,903,Married or in-union women,1992,15-49,66.074066,205592.479669,Estimate
5804,Eastern Africa,910,Married or in-union women,2047,35-39,78.312850,43849.363835,Projection
7389,Southern Africa,913,Married or in-union women,2002,40-44,63.151416,2069.003309,Estimate
15924,Latin America and the Caribbean,904,Married or in-union women,2016,35-39,71.795045,33043.500098,Estimate
17105,Central America,916,Married or in-union women,2002,20-24,49.111576,6577.512833,Estimate


In [865]:
df_15.dropna(inplace=True)
df_15.isnull().sum()

region        0
iso_code      0
indicator     0
year          0
age_group     0
percentage    0
number        0
process       0
dtype: int64

In [866]:
print(df_15['number'] % 1 != 0)

0        True
1        True
2        True
3        True
4        True
         ... 
28507    True
28508    True
28509    True
28510    True
28511    True
Name: number, Length: 28512, dtype: bool


In [867]:
df_15['percentage'] = df_15['percentage'].round(2)
df_15['number'] = df_15['number'].astype(int)
df_15.head(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
0,World,900,Married or in-union women,1970,15-19,22.58,71867,Estimate
1,World,900,Married or in-union women,1970,20-24,63.80,162860,Estimate
2,World,900,Married or in-union women,1970,25-29,87.17,182681,Estimate
3,World,900,Married or in-union women,1970,30-34,90.83,179121,Estimate
4,World,900,Married or in-union women,1970,35-39,90.28,161526,Estimate
5,World,900,Married or in-union women,1970,40-44,86.48,139334,Estimate
6,World,900,Married or in-union women,1970,45-49,82.68,116088,Estimate
7,World,900,Married or in-union women,1970,15-49,69.38,1013479,Estimate
8,World,900,Married or in-union women,1971,15-19,22.63,74127,Estimate
9,World,900,Married or in-union women,1971,20-24,63.61,170087,Estimate


In [868]:
#df_15.to_csv('cleaned_regions_un.csv', index=False)



In [869]:
#df_15.to_sql('regions_un', engine, if_exists='replace',index=False)

In [870]:
df_16_1 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa1.csv')
df_16_1
#Data for Chart SF1.1.A. Average size of households by household type, 2024a
# avg_size_all	avg_size_couple_with_children	avg_size_single_parent_with_children		

,Country,All households,Couple households with children,Single parent households with children
0,Mexico,"3,56","4,08","2,76"
1,Costa Rica,"3,46","4,37","3,44"
2,Türkiye,"3,20","4,10","2,80"
3,Israel,"3,19","4,65","2,86"
4,Columbia,"3,10",NaN,NaN
5,Slovak Republic,"3,10","3,80","2,50"
6,Chile,"2,80",NaN,NaN
7,Iceland,"2,70","4,12","2,61"
8,New Zealand,"2,61","3,88","2,67"
9,Greece,"2,60","3,80","2,50"


In [871]:
df_16_1.columns = df_16_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [872]:
df_16_1.rename(columns={
        "All households": "avg_size_all",
        "Couple with children": "avg_size_couple_with_children",
        "Single parent with children": "avg_size_single_parent_with_children"
}, inplace=True)

In [873]:
df_16_1.drop_duplicates(inplace=True)
df_16_1.dropna(inplace=True)

In [874]:
for col in df_16_1.columns:
    if col != 'country':
        # Replace commas with dots if necessary, remove non-numeric chars, convert to float
        df_16_1[col] = (
            df_16_1[col]
            .astype(str)  # ensure string for replace
            .str.replace(',', '.', regex=False)  # decimal commas to dots
            .str.replace(r'[^\d\.\-]', '', regex=True)  # remove non-numeric chars except dot and minus
            .replace('', None)  # empty to NaN
            .astype(float)  # convert to float
        )

# Check updated dtypes
print(df_16_1.dtypes)

country                                    object
all_households                            float64
couple_households_with_children           float64
single_parent_households_with_children    float64
dtype: object


In [875]:
info_16_1 = pd.DataFrame({
    'dtype': df_16_1.dtypes,
    'null_count': df_16_1.isnull().sum(),
    'unique_count': df_16_1.nunique()
})
print(info_16_1)

                                          dtype  null_count  unique_count
country                                  object           0            39
all_households                          float64           0            19
couple_households_with_children         float64           0            16
single_parent_households_with_children  float64           0            15


In [876]:
df_16_1.sample(10)

,country,all_households,couple_households_with_children,single_parent_households_with_children
42,Finland,1.90,4.00,2.60
28,Switzerland,2.21,4.02,2.58
26,United Kingdom,2.30,3.90,2.80
30,Japan,2.21,3.85,2.73
13,Spain,2.50,3.70,2.40
9,Greece,2.60,3.80,2.50
8,New Zealand,2.61,3.88,2.67
29,Korea,2.21,3.55,2.34
40,Norway,2.00,3.90,2.40
31,Austria,2.20,3.80,2.50


In [877]:
#df_16_1.to_csv('../data/Cleaned/cleaned_average_size_of_households_type_2024_oecd.csv', index=False)

In [878]:
#df_16_1.to_sql('average_size_of_households_type_2024_oecd', engine, if_exists = 'replace', index= False)

In [879]:
df_16_2 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa2.csv', header=1)
df_16_2
#Table SF1.1.A. Types of household, 2021a
# share_couple_total	share_couple_with_children	share_couple_without_children	share_single_parent_total	share_single_mother	share_single_father	share_single_person	share_other_types						

,Country,Total,With children,Without children,Total.1,Single mother households,Single father households,Single person households,Other households types
0,Australia,"55,93","29,90","26,02","10,37",NaN,NaN,"25,12","8,58"
1,Austria,"48,93","21,13","27,80","5,63","4,78","0,85","38,34","7,11"
2,Belgium,"52,22","23,98","28,24","7,42","6,08","1,35","35,50","4,86"
3,Canada,"50,92","25,30","25,62","8,72",NaN,NaN,"29,35","11,02"
4,Chile,..,..,..,..,..,..,..,..
5,Columbia,..,..,..,..,..,..,..,..
6,Costa Rica,"52,44","38,15","14,29","10,55","9,49","1,06","11,27","25,74"
7,Czechia,"47,03","21,70","25,32","7,15","6,11","1,04","39,15","6,67"
8,Denmark,"48,60","20,41","28,19","6,31","5,11","1,19","37,57","7,52"
9,Estonia,"46,20","25,46","20,73","6,83","6,09","0,74","36,99","9,98"


In [880]:
df_16_2.rename(columns={
    "Total": "couple_total(%)",
    "Couple with children": "couple_with_children(%)",
    "Couple without children": "couple_without_children(%)",
    "Total.1": "single_parent_total(%)",
    "Single mother households": "single_mother(%)",
    "Single father households": "single_father(%)",
    "Single person households": "single_person(%)",
    "Other types of households": "other_household_types(%)"
}, inplace=True)

In [881]:
# Normalize column names (lowercase, underscores, remove () and non-ascii)
df_16_2.columns = (
    df_16_2.columns.str.strip().str.lower()
    .str.replace(' ', '_')
    .str.replace('[()%]', '', regex=True)
    .str.replace('[^0-9a-z_]', '', regex=True)
)

In [882]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_16_2.columns if c != "country"]

df_16_2[num_cols] = (
    df_16_2[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [883]:
df_16_2.drop_duplicates(inplace=True)
df_16_2.dropna(inplace=True)
df_16_2.dropna(how="all", subset=num_cols, inplace=True)

In [884]:
df_16_2.rename(columns={
   "couple_total" : "couple_total(%)",
   "with_children" : "with_children(%)",
   "without_children" : "without_children(%)",
    "single_parent_total" : "single_parent_total(%)",
    "single_mother" : "single_mother(%)",
    "single_father" : "single_father(%)",
    "single_person" : "single_person(%)",
    "other_household_types" : "other_household_types(%)"
}, inplace=True)

In [885]:
info_16_2 = pd.DataFrame({
    "dtype": df_16_2.dtypes,
    "null_count": df_16_2.isna().sum(),
    "unique_count": df_16_2.nunique()
})
print(info_16_2)
print(df_16_2.dtypes)

                          dtype  null_count  unique_count
country                  object           0            36
couple_total(%)         float64           0            36
with_children(%)        float64           0            35
without_children(%)     float64           0            36
single_parent_total(%)  float64           0            34
single_mother(%)        float64           0            32
single_father(%)        float64           0            31
single_person(%)        float64           0            35
other_households_types  float64           0            36
country                    object
couple_total(%)           float64
with_children(%)          float64
without_children(%)       float64
single_parent_total(%)    float64
single_mother(%)          float64
single_father(%)          float64
single_person(%)          float64
other_households_types    float64
dtype: object


In [886]:
df_16_2.sample(10)

,country,couple_total(%),with_children(%),without_children(%),single_parent_total(%),single_mother(%),single_father(%),single_person(%),other_households_types
19,Japan,45.07,15.67,29.40,2.50,2.26,0.25,37.97,14.45
18,Italy,46.70,20.91,25.80,7.27,5.65,1.63,36.64,9.38
39,Bulgaria,40.30,16.35,23.95,4.60,3.88,0.73,35.81,19.28
25,Netherlands,53.60,23.01,30.59,6.10,5.00,1.09,38.50,1.80
40,Croatia,51.51,24.78,26.73,5.42,4.39,1.04,27.80,15.27
41,Cyprus,56.92,27.42,29.50,6.17,4.94,1.23,24.49,12.43
33,Sweden,49.27,22.49,26.78,6.67,4.91,1.76,39.24,4.82
42,Malta,46.92,21.11,25.81,5.68,4.56,1.12,32.51,14.89
30,Slovak Republic,37.15,16.99,20.16,6.23,5.39,0.84,31.40,25.21
10,Finland,45.64,17.06,28.58,5.43,4.50,0.93,45.34,3.60


In [887]:
#df_16_2.to_csv('../data/Cleaned/cleaned_types_of_household_2021_oecd.csv', index = False)

In [888]:
#df_16_2.to_sql('types_of_household_2021_oecd', engine, if_exists = 'replace', index= False)

In [889]:
df_16_3 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa3.csv', header=1)
df_16_3
#Table SF1.1.B. Households by number of children, 2024
# share_hh_0_children	share_hh_1_child	share_hh_2_children	share_hh_3plus_children		

,country,0 children,1 child,2 children,3 or more children,Children under 6
0,Australia,..,..,..,..,..
1,Austria,"77,78","10,52","8,57","3,12","9,44"
2,Belgium,"73,97","11,76","10,15","4,11","10,40"
3,Canada,..,..,..,..,..
4,Chile,..,..,..,..,..
5,Columbia,..,..,..,..,..
6,Costa Rica,"30,29","23,08","24,61","22,02","26,30"
7,Czechia,"71,95","13,85","11,56","2,64","12,29"
8,Denmark,"77,78","10,54","8,94","2,74","8,15"
9,Estonia,"75,76","12,53","8,73","2,98","9,85"


In [890]:
# Normalize column names (lowercase, underscores, remove () and non-ascii)
df_16_3.columns = (
    df_16_3.columns.str.strip().str.lower()
    .str.replace(' ', '_')
    .str.replace('[^0-9a-z_]', '', regex=True)
)

In [891]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_16_3.columns if c != "country"]

df_16_3[num_cols] = (
    df_16_3[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [892]:
df_16_3.drop_duplicates(inplace=True)
df_16_3.dropna(inplace=True)

In [893]:
df_16_3.sample(10)

,country,0_children,1_child,2_children,3_or_more_children,children_under_6
9,Estonia,75.76,12.53,8.73,2.98,9.85
21,Latvia,74.80,14.05,8.32,2.83,10.07
16,Ireland,69.02,12.42,12.18,6.38,11.81
30,Slovak Republic,64.41,17.09,14.49,4.02,15.56
14,Hungary,74.98,13.16,8.71,3.16,10.41
13,Greece,74.31,11.83,9.97,3.89,9.37
8,Denmark,77.78,10.54,8.94,2.74,8.15
25,Netherlands,78.65,8.78,9.27,3.30,8.79
22,Lithuania,80.44,11.06,7.00,1.51,8.12
1,Austria,77.78,10.52,8.57,3.12,9.44


In [894]:
df_16_3.rename(columns={
    "0_children": "households_0_children(%)",
    "1_child": "households_1_child(%)",
    "2_children": "households_2_children(%)",
    "3_or_more_children": "households_3_or_more_children(%)"
}, inplace=True)

In [895]:
info_16_3 = pd.DataFrame({
    "dtype": df_16_3.dtypes,
    "null_count": df_16_3.isna().sum(),
    "unique_count": df_16_3.nunique()
})
print(info_16_3)
print(df_16_3.dtypes)

                                    dtype  null_count  unique_count
country                            object           0            33
households_0_children(%)          float64           0            32
households_1_child(%)             float64           0            32
households_2_children(%)          float64           0            33
households_3_or_more_children(%)  float64           0            31
children_under_6                  float64           0            31
country                              object
households_0_children(%)            float64
households_1_child(%)               float64
households_2_children(%)            float64
households_3_or_more_children(%)    float64
children_under_6                    float64
dtype: object


In [896]:
df_16_3.sample(10)

,country,households_0_children(%),households_1_child(%),households_2_children(%),households_3_or_more_children(%),children_under_6
22,Lithuania,80.44,11.06,7.00,1.51,8.12
8,Denmark,77.78,10.54,8.94,2.74,8.15
10,Finland,81.98,7.89,6.99,3.14,7.14
1,Austria,77.78,10.52,8.57,3.12,9.44
41,Cyprus,71.36,13.88,11.67,3.10,12.71
28,Poland,74.39,12.91,9.84,2.86,9.97
19,Japan,81.94,8.78,7.17,2.11,7.58
32,Spain,74.61,13.54,8.95,2.90,8.79
14,Hungary,74.98,13.16,8.71,3.16,10.41
33,Sweden,74.84,10.77,9.83,4.56,9.95


In [897]:
#df_16_3.to_csv('../data/Cleaned/cleaned_households_by_number_of_children_2024_oecd.csv', index=False)

In [898]:
#df_16_3.to_sql('households_by_number_of_children_2024_oecd', engine, index= False)

In [899]:
df_17_1 = pd.read_csv('../data/Raw/OECD/SF_2_1_Total_Fertility_rates_S1.csv')
#total_fertility_rates_from_1960_oecd
df_17_1.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Australia,"3,45","3,55","3,43","3,34","3,15","2,97","2,89","2,85","2,89",...,"1,79","1,79","1,79","1,74","1,74","1,67","1,59","1,70","1,63","1,50"
1,Austria,"2,69","2,78","2,80","2,82","2,79","2,70","2,66","2,62","2,58",...,"1,46","1,49","1,53","1,52","1,48","1,46","1,44","1,48","1,41","1,32"
2,Belgium,"2,54","2,63","2,59","2,68","2,71","2,61","2,52","2,41","2,31",...,"1,74","1,70","1,68","1,65","1,62","1,60","1,55","1,60","1,53","1,47"
3,Canada,"3,90","3,84","3,76","3,67","3,50","3,15","2,81","2,60","2,45",...,"1,61","1,60","1,59","1,55","1,51","1,47","1,41","1,44","1,33","1,26"
4,Chile,"4,70","4,66","4,60","4,54","4,46","4,36","4,26","4,14","4,03",...,"1,77","1,74","1,69","1,56","1,54","1,43","1,31","1,18","1,26","1,17"


In [900]:
df_info = pd.DataFrame({
    'dtype': df_17_1.dtypes,
    'null_count': df_17_1.isnull().sum(),
    'unique_count': df_17_1.nunique()
})
print(df_info)

          dtype  null_count  unique_count
Country  object           0            49
1960     object           0            47
1961     object           0            47
1962     object           0            47
1963     object           0            46
...         ...         ...           ...
2019     object           0            37
2020     object           0            39
2021     object           0            40
2022     object           0            34
2023     object           0            35

[65 rows x 3 columns]


In [901]:
df_17_1.columns = df_17_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [902]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_17_1.columns if c != "country"]

df_17_1[num_cols] = (
    df_17_1[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [903]:
df_17_1.drop_duplicates(inplace=True)
df_17_1.dropna(inplace=True)

In [904]:
df_info = pd.DataFrame({
    'dtype': df_17_1.dtypes,
    'null_count': df_17_1.isnull().sum(),
    'unique_count': df_17_1.nunique()
})
print(df_info)

           dtype  null_count  unique_count
country   object           0            49
1960     float64           0            47
1961     float64           0            47
1962     float64           0            47
1963     float64           0            46
...          ...         ...           ...
2019     float64           0            37
2020     float64           0            39
2021     float64           0            40
2022     float64           0            34
2023     float64           0            35

[65 rows x 3 columns]


In [905]:
df_17_1.sample(10)

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,Austria,2.69,2.78,2.80,2.82,2.79,2.70,2.66,2.62,2.58,...,1.46,1.49,1.53,1.52,1.48,1.46,1.44,1.48,1.41,1.32
20,Latvia,1.94,1.94,1.91,1.85,1.79,1.74,1.76,1.80,1.83,...,1.65,1.70,1.74,1.69,1.60,1.61,1.55,1.57,1.47,1.36
4,Chile,4.70,4.66,4.60,4.54,4.46,4.36,4.26,4.14,4.03,...,1.77,1.74,1.69,1.56,1.54,1.43,1.31,1.18,1.26,1.17
27,Poland,2.98,2.83,2.72,2.70,2.57,2.52,2.34,2.33,2.24,...,1.29,1.29,1.36,1.45,1.44,1.42,1.39,1.33,1.26,1.16
25,New Zealand,4.24,4.31,4.19,4.05,3.80,3.54,3.41,3.35,3.34,...,1.92,1.99,1.87,1.81,1.71,1.72,1.61,1.64,1.66,1.56
33,Switzerland,2.44,2.53,2.60,2.67,2.68,2.61,2.52,2.41,2.30,...,1.54,1.54,1.54,1.52,1.52,1.48,1.46,1.52,1.39,1.33
2,Belgium,2.54,2.63,2.59,2.68,2.71,2.61,2.52,2.41,2.31,...,1.74,1.70,1.68,1.65,1.62,1.60,1.55,1.60,1.53,1.47
11,France,2.74,2.82,2.80,2.90,2.91,2.85,2.80,2.67,2.59,...,2.00,1.96,1.92,1.89,1.87,1.86,1.82,1.83,1.78,1.66
10,Finland,2.71,2.65,2.66,2.66,2.58,2.46,2.41,2.32,2.15,...,1.71,1.65,1.57,1.49,1.41,1.35,1.37,1.46,1.32,1.26
3,Canada,3.90,3.84,3.76,3.67,3.50,3.15,2.81,2.60,2.45,...,1.61,1.60,1.59,1.55,1.51,1.47,1.41,1.44,1.33,1.26


In [906]:
#df_17_1.to_csv('../data/Cleaned/cleaned_total_fertility_rates_oecd.csv', index=False)

In [907]:
#df_17_1.to_sql('total_fertility_rates_oecd', engine, if_exists='replace', index=False)

In [908]:
df_17_2 = pd.read_csv('../data/Raw/OECD/SF_2_1_Fertility_rates_Births_by_birth_order_S2.csv')
df_17_2

,Country,Birth order,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,First birth,"47,6","47,8","46,7","46,2","46,5","46,1","45,2","44,5",...,"48,0","48,3","47,3","47,5","47,1","47,2","47,7","47,6","48,4","48,1"
1,Austria,Second birth,"33,7","33,7","34,3","34,9","34,5","34,8","35,8","36,4",...,"35,5","35,3","35,6","35,3","35,3","35,1","35,3","35,5","34,9","35,1"
2,Austria,Third birth or higher,"18,8","18,5","19,0","18,9","19,0","19,1","18,9","19,1",...,"16,5","16,4","17,1","17,2","17,6","17,7","17,0","16,9","16,7","16,8"
3,Belgium,First birth,"46,8","46,9","47,3","47,3","48,1","47,2","46,9","47,2",...,"42,3","43,5","44,1","43,6","42,9","42,6","45,0","44,0","44,7","45,5"
4,Belgium,Second birth,"33,0","32,9","32,7","32,8","32,3","32,8","33,5","33,0",...,"35,1","34,8","34,5","34,6","34,5","34,7","34,2","35,1","34,3","34,1"
5,Belgium,Third birth or higher,"20,2","20,2","19,9","19,9","19,6","20,0","19,6","19,8",...,"22,6","21,8","21,4","21,9","22,6","22,6","20,8","20,9","20,9","20,4"
6,Czechia,First birth,"46,7","46,6","47,4","47,8","50,1","49,8","48,5","47,7",...,"47,4","48,1","48,7","48,7","48,0","47,8","47,6","46,4","46,3","46,3"
7,Czechia,Second birth,"37,7","37,6","37,4","37,2","35,5","35,8","36,8","36,9",...,"37,5","37,3","36,7","36,6","37,2","37,6","37,6","39,0","38,6","39,1"
8,Czechia,Third birth or higher,"15,6","15,8","15,2","15,0","14,4","14,4","14,8","15,4",...,"15,1","14,7","14,6","14,7","14,7","14,6","14,8","14,6",15,"14,6"
9,Estonia,First birth,"43,5","43,5","44,0","46,2","49,5","50,3","49,6","49,6",...,"41,9","42,3","40,8","40,2","36,7","38,8","38,0","37,2","39,8","39,7"


In [909]:
df_info = pd.DataFrame({
    'dtype': df_17_2.dtypes,
    'null_count': df_17_2.isnull().sum(),
    'unique_count': df_17_2.nunique()
})
print(df_info)

              dtype  null_count  unique_count
Country      object           0            17
Birth order  object           0             3
1987         object           0            48
1988         object           0            49
1989         object           0            48
1990         object           0            44
1991         object           0            48
1992         object           0            46
1993         object           0            47
1994         object           0            47
1995         object           0            48
1996         object           0            47
1997         object           0            49
1998         object           0            50
1999         object           0            49
2000         object           0            48
2001         object           0            50
2002         object           0            47
2003         object           0            50
2004         object           0            49
2005         object           0   

In [910]:
df_17_2.columns = df_17_2.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_17_2.head()

,country,birth_order,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,First birth,"47,6","47,8","46,7","46,2","46,5","46,1","45,2","44,5",...,"48,0","48,3","47,3","47,5","47,1","47,2","47,7","47,6","48,4","48,1"
1,Austria,Second birth,"33,7","33,7","34,3","34,9","34,5","34,8","35,8","36,4",...,"35,5","35,3","35,6","35,3","35,3","35,1","35,3","35,5","34,9","35,1"
2,Austria,Third birth or higher,"18,8","18,5","19,0","18,9","19,0","19,1","18,9","19,1",...,"16,5","16,4","17,1","17,2","17,6","17,7","17,0","16,9","16,7","16,8"
3,Belgium,First birth,"46,8","46,9","47,3","47,3","48,1","47,2","46,9","47,2",...,"42,3","43,5","44,1","43,6","42,9","42,6","45,0","44,0","44,7","45,5"
4,Belgium,Second birth,"33,0","32,9","32,7","32,8","32,3","32,8","33,5","33,0",...,"35,1","34,8","34,5","34,6","34,5","34,7","34,2","35,1","34,3","34,1"


In [911]:
# --- Ensure "country" and "birth order" are strings (tidy casing/spacing) ---
df_17_2["country"] = df_17_2["country"].astype(str).str.strip().str.title()
df_17_2["birth_order"] = df_17_2["birth_order"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_17_2.columns if c not in ["country", "birth_order"]]
# --- Robust cleaning -> convert to float ---
df_17_2[num_cols] = (
    df_17_2[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_17_2[num_cols] = df_17_2[num_cols].round(2)


In [912]:
df_17_2.drop_duplicates(inplace=True)
df_17_2.dropna(inplace=True)

In [913]:
df_17_2["birth_order"] = df_17_2["birth_order"].astype(str) + "(%)"

In [914]:
df_17_2.head(10)

,country,birth_order,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,First birth(%),47.6,47.8,46.7,46.2,46.5,46.1,45.2,44.5,...,48.0,48.3,47.3,47.5,47.1,47.2,47.7,47.6,48.4,48.1
1,Austria,Second birth(%),33.7,33.7,34.3,34.9,34.5,34.8,35.8,36.4,...,35.5,35.3,35.6,35.3,35.3,35.1,35.3,35.5,34.9,35.1
2,Austria,Third birth or higher(%),18.8,18.5,19.0,18.9,19.0,19.1,18.9,19.1,...,16.5,16.4,17.1,17.2,17.6,17.7,17.0,16.9,16.7,16.8
3,Belgium,First birth(%),46.8,46.9,47.3,47.3,48.1,47.2,46.9,47.2,...,42.3,43.5,44.1,43.6,42.9,42.6,45.0,44.0,44.7,45.5
4,Belgium,Second birth(%),33.0,32.9,32.7,32.8,32.3,32.8,33.5,33.0,...,35.1,34.8,34.5,34.6,34.5,34.7,34.2,35.1,34.3,34.1
5,Belgium,Third birth or higher(%),20.2,20.2,19.9,19.9,19.6,20.0,19.6,19.8,...,22.6,21.8,21.4,21.9,22.6,22.6,20.8,20.9,20.9,20.4
6,Czechia,First birth(%),46.7,46.6,47.4,47.8,50.1,49.8,48.5,47.7,...,47.4,48.1,48.7,48.7,48.0,47.8,47.6,46.4,46.3,46.3
7,Czechia,Second birth(%),37.7,37.6,37.4,37.2,35.5,35.8,36.8,36.9,...,37.5,37.3,36.7,36.6,37.2,37.6,37.6,39.0,38.6,39.1
8,Czechia,Third birth or higher(%),15.6,15.8,15.2,15.0,14.4,14.4,14.8,15.4,...,15.1,14.7,14.6,14.7,14.7,14.6,14.8,14.6,15.0,14.6
9,Estonia,First birth(%),43.5,43.5,44.0,46.2,49.5,50.3,49.6,49.6,...,41.9,42.3,40.8,40.2,36.7,38.8,38.0,37.2,39.8,39.7


In [915]:
df_info = pd.DataFrame({
    'dtype': df_17_2.dtypes,
    'null_count': df_17_2.isnull().sum(),
    'unique_count': df_17_2.nunique()
})
print(df_info)

               dtype  null_count  unique_count
country       object           0            17
birth_order   object           0             3
1987         float64           0            48
1988         float64           0            49
1989         float64           0            48
1990         float64           0            44
1991         float64           0            48
1992         float64           0            46
1993         float64           0            47
1994         float64           0            47
1995         float64           0            48
1996         float64           0            47
1997         float64           0            49
1998         float64           0            50
1999         float64           0            49
2000         float64           0            48
2001         float64           0            50
2002         float64           0            47
2003         float64           0            50
2004         float64           0            49
2005         

In [916]:
#df_17_2.to_csv('../data/Cleaned/cleaned_births_by_birth_order_oecd.csv', index=False)

In [917]:
#df_17_2.to_sql('births_by_birth_order_oecd', engine, if_exists='replace', index=False)

In [918]:
df_18 = pd.read_csv('../data/Raw/OECD/sf1_2_wide_from_df18.csv')
df_18

,country,year,Living with two parents,Living with a single parent,Other
0,Australia,2003,80.1,19.5,0.5
1,Australia,2006,81.5,18.0,0.5
2,Australia,2009,82.0,17.6,0.4
3,Australia,2012,81.3,18.0,0.6
4,Austria,2003,81.2,16.8,2.0
...,...,...,...,...,...
470,United States,2014,68.7,27.5,3.8
471,United States,2015,69.2,26.8,3.9
472,United States,2016,68.7,27.4,3.8
473,United States,2017,68.9,27.1,4.0


In [919]:
for col in df_18.select_dtypes(include=['object']).columns:
    df_18[col] = df_18[col].astype(str).str.strip()

# 2) Define placeholders representing missing data in OECD exports
placeholders = ['..', '...', '.', ' .', '…', 'Na', 'nan', 'None']

# 3) Replace placeholders with NaN directly in df_18
df_18.replace(placeholders, pd.NA, inplace=True)

In [920]:
# 1) Ensure 'year' is integer
df_18["year"] = pd.to_numeric(df_18["year"], errors="coerce").astype("Int64")

# 2) Convert all non-key columns to numeric and round(2)
for col in df_18.columns:
    if col not in ["country", "year"]:
        df_18[col] = pd.to_numeric(df_18[col], errors="coerce").round(2)

In [921]:
# 1) Drop rows with missing key fields
df_18.dropna(subset=["country", "year"], inplace=True)

# 2) Drop duplicate country-year rows, keep the first
df_18.drop_duplicates(subset=["country", "year"], keep="first", inplace=True)

# 3) Drop rows where all value columns are NaN
value_cols = [c for c in df_18.columns if c not in ["country", "year"]]
df_18.dropna(subset=value_cols, how="all", inplace=True)

# 4) Sort and reset index
df_18.sort_values(["country", "year"], inplace=True)
df_18.reset_index(drop=True, inplace=True)


In [922]:
df_18

,country,year,Living with two parents,Living with a single parent,Other
0,Australia,2003,80.1,19.5,0.5
1,Australia,2006,81.5,18.0,0.5
2,Australia,2009,82.0,17.6,0.4
3,Australia,2012,81.3,18.0,0.6
4,Austria,2003,81.2,16.8,2.0
...,...,...,...,...,...
470,United States,2014,68.7,27.5,3.8
471,United States,2015,69.2,26.8,3.9
472,United States,2016,68.7,27.4,3.8
473,United States,2017,68.9,27.1,4.0


In [923]:
df_18['Other'] = pd.to_numeric(df_18['Other'], errors='coerce')

In [924]:
df_info = pd.DataFrame({
    'dtype': df_18.dtypes,
    'null_count': df_18.isnull().sum(),
    'unique_count': df_18.nunique()
})
print(df_info)

                               dtype  null_count  unique_count
country                       object           0            39
year                           Int64           0            18
Living with two parents      float64           0           211
Living with a single parent  float64           0           203
Other                        float64           1            50


In [925]:
print(repr(df_18.loc[df_18['Other'].notnull(), 'Other'].unique()))

array([0.5, 0.4, 0.6, 2. , 1. , 1.9, 0.3, 0.1, 0.8, 0.7, 8.7, 3.5, 2.5,
       2.1, 2.4, 2.6, 6.7, 5.1, 1.4, 1.2, 1.7, 1.5, 3.4, 2.9, 2.3, 3. ,
       4.2, 2.8, 1.3, 9. , 0.2, 0.9, 1.1, 4.5, 4.7, 1.6, 3.8, 3.6, 3.3,
       2.2, 0. , 1.8, 2.7, 3.2, 3.9, 4.1, 4.4, 3.7, 4. , 4.3])


In [926]:
df_18['Other'] = pd.to_numeric(df_18['Other'], errors='coerce')

df_18.dropna(inplace=True, subset=['Other'])

df_18.isnull().sum()

country                        0
year                           0
Living with two parents        0
Living with a single parent    0
Other                          0
dtype: int64

In [927]:
#df_18.to_csv('../data/Cleaned/cleaned_household_children.csv', index=False)

In [928]:
#df_18.to_sql('household_children_oecd', engine, if_exists= 'replace', index= False)

In [929]:
df_19_1 =pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_mean_age_birth_S1.csv')
#age_of_mothers_at_childbirth
df_19_1

,Country,1963,1964,1965,1966,1967,1968,1969,1970,1971,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,"27,5","27,5","27,4","27,3","27,3","27,2","27,2","27,1","26,9",...,"30,1","30,1","30,2","30,3","30,5","30,6","30,7","30,8","30,9","31,1"
1,Austria,"27,4","27,4","27,3","27,1","27,0","26,8","26,8","26,7","26,7",...,"30,2","30,3","30,4","30,6","30,6","30,7","30,9","31,0","31,0","31,2"
2,Belgium,"27,8","27,7","27,6","27,5","27,4","27,3","27,2","27,2","27,0",...,"30,0","30,2","30,3","30,4","30,5","30,6","30,7","30,8","30,8","31,0"
3,Canada,"27,8","27,9","27,8","27,7","27,5","27,3","27,3","27,2","27,0",...,"30,3","30,4","30,5","30,6","30,7","30,9","31,0","31,2","31,3","31,4"
4,Chile,"29,2","29,1","29,1","29,0","28,8","28,7","28,6","28,4","28,2",...,"28,1","28,3","28,5","28,8","29,1","29,4","29,6","29,9","30,1",..
5,Czech Republic,"25,7","25,8","25,5","25,2","25,0","24,9","24,8","24,8","24,9",...,"29,8","29,9","29,9","30,0","30,0","30,0","30,1","30,2","30,2","30,4"
6,Costa Rica,"29,3","29,3","29,3","29,3","29,2","29,1","28,9","28,7","28,5",...,"26,5","26,7","26,8","27,1","27,2","27,4","27,6","27,9","28,4","28,7"
7,Denmark,"27,3","26,8","26,8","26,6","26,5","26,5","26,6","26,7","26,7",...,"30,7","30,8","30,9","31,0","31,0","31,1","31,2","31,3","31,4","31,6"
8,Estonia,"27,6","27,4","27,3","27,3","27,1","26,9","26,9","26,7","26,7",...,"29,6","29,5","29,6","29,9","30,2","30,4","30,5","30,6","30,7","31,0"
9,Finland,"28,1","28,0","28,0","27,8","27,7","27,5","27,4","27,1","26,9",...,"30,4","30,5","30,5","30,6","30,8","30,9","31,0","31,1","31,2","31,4"


In [930]:
df_info = pd.DataFrame({
    'dtype': df_19_1.dtypes,
    'null_count': df_19_1.isnull().sum(),
    'unique_count': df_19_1.nunique()
})
print(df_info)

          dtype  null_count  unique_count
Country  object           0            26
1963     object           0            19
1964     object           0            22
1965     object           0            22
1966     object           0            22
1967     object           0            22
1968     object           0            20
1969     object           0            21
1970     object           0            19
1971     object           0            19
1972     object           0            20
1973     object           0            20
1974     object           0            24
1975     object           0            21
1976     object           0            22
1977     object           0            20
1978     object           0            22
1979     object           0            23
1980     object           0            22
1981     object           0            20
1982     object           0            18
1983     object           0            20
1984     object           0       

In [931]:
df_19_1.columns = df_19_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [932]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_19_1.columns if c != "country"]

df_19_1[num_cols] = (
    df_19_1[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [933]:
df_19_1.drop_duplicates(inplace=True)
df_19_1.dropna(inplace=True)

In [934]:
df_info = pd.DataFrame({
    'dtype': df_19_1.dtypes,
    'null_count': df_19_1.isnull().sum(),
    'unique_count': df_19_1.nunique()
})
print(df_info)

           dtype  null_count  unique_count
country   object           0            22
1963     float64           0            16
1964     float64           0            18
1965     float64           0            18
1966     float64           0            18
1967     float64           0            18
1968     float64           0            17
1969     float64           0            17
1970     float64           0            15
1971     float64           0            17
1972     float64           0            18
1973     float64           0            18
1974     float64           0            20
1975     float64           0            18
1976     float64           0            18
1977     float64           0            16
1978     float64           0            18
1979     float64           0            21
1980     float64           0            20
1981     float64           0            17
1982     float64           0            17
1983     float64           0            18
1984     fl

In [935]:
df_19_1.sample(10)

,country,1963,1964,1965,1966,1967,1968,1969,1970,1971,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
21,Slovak Republic,26.8,26.9,26.8,26.6,26.5,26.4,26.3,26.2,26.2,...,28.7,28.8,28.8,28.8,28.8,28.8,28.8,28.8,28.9,28.9
2,Belgium,27.8,27.7,27.6,27.5,27.4,27.3,27.2,27.2,27.0,...,30.0,30.2,30.3,30.4,30.5,30.6,30.7,30.8,30.8,31.0
8,Estonia,27.6,27.4,27.3,27.3,27.1,26.9,26.9,26.7,26.7,...,29.6,29.5,29.6,29.9,30.2,30.4,30.5,30.6,30.7,31.0
0,Australia,27.5,27.5,27.4,27.3,27.3,27.2,27.2,27.1,26.9,...,30.1,30.1,30.2,30.3,30.5,30.6,30.7,30.8,30.9,31.1
5,Czech Republic,25.7,25.8,25.5,25.2,25.0,24.9,24.8,24.8,24.9,...,29.8,29.9,29.9,30.0,30.0,30.0,30.1,30.2,30.2,30.4
14,Israel,28.1,28.1,28.2,28.2,28.2,28.5,28.4,28.2,28.5,...,30.1,30.2,30.3,30.3,30.4,30.4,30.4,30.5,30.5,30.6
1,Austria,27.4,27.4,27.3,27.1,27.0,26.8,26.8,26.7,26.7,...,30.2,30.3,30.4,30.6,30.6,30.7,30.9,31.0,31.0,31.2
11,Hungary,25.8,25.7,25.6,25.6,25.6,25.5,25.5,25.4,25.4,...,29.4,29.5,29.5,29.6,29.6,29.8,29.8,29.9,29.9,30.0
7,Denmark,27.3,26.8,26.8,26.6,26.5,26.5,26.6,26.7,26.7,...,30.7,30.8,30.9,31.0,31.0,31.1,31.2,31.3,31.4,31.6
19,Norway,27.8,27.8,27.7,27.6,27.5,27.3,27.2,27.0,26.8,...,30.3,30.5,30.6,30.7,30.8,31.0,31.1,31.3,31.4,31.5


In [936]:
#df_19_1.to_csv('../data/Cleaned/age_of_mothers_at_childbirth_oecd.csv', index=False)

In [937]:
#df_19_1.to_sql('age_of_mothers_at_childbirth_oecd', engine, if_exists='replace', index=False)

In [938]:
df_19_2 = pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_fertility_by_age_1960_S2.csv')
#fertility_per_1000_from 1960
df_19_2.head()

,Country,Age group,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,15-19,"44,3","47,4","44,7","45,9","47,0","47,5","48,9","48,4",...,"16,1","14,6","12,9","12,0","10,5","10,3","9,5","8,8","7,9","7,1"
1,Australia,20-24,"220,1","225,8","216,0","208,2","190,5","179,3","173,1","170,8",...,"53,2","51,3","47,4","47,3","44,7","43,1","42,8","40,1","37,7","38,8"
2,Australia,25-29,"216,3","221,2","216,7","211,2","198,1","188,5","183,9","185,0",...,"102,6","99,1","94,8","93,4","92,2","89,7","89,3","84,3","80,3","86,7"
3,Australia,30-34,"127,5","131,1","127,7","123,9","119,1","110,1","105,1","102,8",...,"126,9","124,8","120,4","121,7","123,6","119,1","120,1","115,6","111,4","120,6"
4,Australia,35-39,"62,3","63,4","61,4","59,7","58,4","53,0","50,6","47,8",...,"71,5","70,9","69,2","69,8","72,0","71,3","71,6","69,3","66,3","70,9"


In [939]:
df_info = pd.DataFrame({
    'dtype': df_19_2.dtypes,
    'null_count': df_19_2.isnull().sum(),
    'unique_count': df_19_2.nunique()
})
print(df_info)

            dtype  null_count  unique_count
Country    object           0            21
Age group  object           0             7
1960       object           0           136
1961       object           0           140
1962       object           0           140
...           ...         ...           ...
2017       object           0           124
2018       object           0           128
2019       object           0           126
2020       object           0           121
2021       object           7           119

[64 rows x 3 columns]


In [940]:
df_19_2.columns = df_19_2.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_19_2.head()

,country,age_group,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,15-19,"44,3","47,4","44,7","45,9","47,0","47,5","48,9","48,4",...,"16,1","14,6","12,9","12,0","10,5","10,3","9,5","8,8","7,9","7,1"
1,Australia,20-24,"220,1","225,8","216,0","208,2","190,5","179,3","173,1","170,8",...,"53,2","51,3","47,4","47,3","44,7","43,1","42,8","40,1","37,7","38,8"
2,Australia,25-29,"216,3","221,2","216,7","211,2","198,1","188,5","183,9","185,0",...,"102,6","99,1","94,8","93,4","92,2","89,7","89,3","84,3","80,3","86,7"
3,Australia,30-34,"127,5","131,1","127,7","123,9","119,1","110,1","105,1","102,8",...,"126,9","124,8","120,4","121,7","123,6","119,1","120,1","115,6","111,4","120,6"
4,Australia,35-39,"62,3","63,4","61,4","59,7","58,4","53,0","50,6","47,8",...,"71,5","70,9","69,2","69,8","72,0","71,3","71,6","69,3","66,3","70,9"


In [941]:
# --- Ensure "country" and "age_group" are strings
df_19_2["country"] = df_19_2["country"].astype(str).str.strip().str.title()
df_19_2["age_group"] = df_19_2["age_group"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_19_2.columns if c not in ["country", "age_group"]]
# --- Robust cleaning -> convert to float ---
df_19_2[num_cols] = (
    df_19_2[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_19_2[num_cols] = df_19_2[num_cols].round(2)

In [942]:
df_19_2.drop_duplicates(inplace=True)
df_19_2.dropna(inplace = True)

In [943]:
df_info = pd.DataFrame({
    'dtype': df_19_2.dtypes,
    'null_count': df_19_2.isnull().sum(),
    'unique_count': df_19_2.nunique()
})
print(df_info)

             dtype  null_count  unique_count
country     object           0            19
age_group   object           0             7
1960       float64           0           124
1961       float64           0           126
1962       float64           0           126
...            ...         ...           ...
2017       float64           0           118
2018       float64           0           121
2019       float64           0           120
2020       float64           0           115
2021       float64           0           118

[64 rows x 3 columns]


In [944]:
#df_19_2.to_csv('../data/Cleaned/fertility_per_1000_by_age_from 1960_oecd.csv', index=False)

In [945]:
#df_19_2.to_sql('fertility_per_1000_from_1960_oecd', engine, if_exists='replace', index=False)

In [946]:
df_19_3 = pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_fertility_by_age_2000_S3.csv')
#fertility_per_1000_from_2000
df_19_3

,Country,Age group,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,OECD-Average,15-19,"22,6","22,0","21,1","20,5","20,3","20,1","20,0","20,5",...,"17,9","16,8","16,2","15,2","14,4","13,5","12,6","11,7","10,2","9,5"
1,OECD-Average,20-24,"71,7","69,3","66,8","65,5","64,7","63,2","62,9","63,0",...,"56,4","53,8","53,3","51,9","50,4","48,8","47,0","45,0","42,0","40,5"
2,OECD-Average,25-29,"107,9","105,0","103,1","103,5","103,4","102,3","102,6","103,4",...,"99,4","96,5","96,9","96,1","94,9","92,8","90,7","88,4","85,5","86,9"
3,OECD-Average,30-34,"88,1","87,2","88,6","91,1","93,4","94,6","97,6","100,0",...,"103,6","101,9","104,0","104,9","105,3","104,1","103,3","101,7","99,6","103,6"
4,OECD-Average,35-39,"38,1","38,6","39,5","40,6","42,2","43,5","45,6","47,7",...,"53,1","53,4","55,1","56,3","57,1","57,0","57,4","57,5","55,9","58,7"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,Romania,25-29,"78,2","77,0","78,6","82,0","84,8","90,8","92,3","93,0",...,"91,8","88,3","94,4","98,9","100,1","109,0","108,3","109,1","109,4","110,9"
297,Romania,30-34,"38,8","38,1","38,8","38,8","41,6","47,5","51,1","54,2",...,"66,6","64,8","71,5","75,4","78,5","86,6","85,9","86,4","87,1","87,5"
298,Romania,35-39,"13,4","13,8","15,2","19,4","23,2","25,1","25,7","24,9",...,"27,3","27,4","29,9","32,1","33,0","36,8","36,7","38,3","40,6","41,1"
299,Romania,40-44,"3,1","3,1","3,0","2,9","3,1","3,1","2,8","3,1",...,"4,9","4,8","5,6","6,1","6,8","7,3","7,8","8,0","8,5","8,2"


In [947]:
df_info = pd.DataFrame({
    'dtype': df_19_3.dtypes,
    'null_count': df_19_3.isnull().sum(),
    'unique_count': df_19_3.nunique()
})
print(df_info)

            dtype  null_count  unique_count
Country    object           0            43
Age group  object           0             7
2000       object           0           233
2001       object           0           248
2002       object           0           240
2003       object           0           239
2004       object           0           245
2005       object           0           240
2006       object           0           239
2007       object           0           242
2008       object           0           252
2009       object           0           251
2010       object           0           239
2011       object           0           235
2012       object           0           242
2013       object           0           234
2014       object           0           238
2015       object           0           237
2016       object           0           248
2017       object           0           236
2018       object           0           245
2019       object           0   

In [948]:
df_19_3.columns = df_19_3.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_19_3.head()

,country,age_group,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,OECD-Average,15-19,"22,6","22,0","21,1","20,5","20,3","20,1","20,0","20,5",...,"17,9","16,8","16,2","15,2","14,4","13,5","12,6","11,7","10,2","9,5"
1,OECD-Average,20-24,"71,7","69,3","66,8","65,5","64,7","63,2","62,9","63,0",...,"56,4","53,8","53,3","51,9","50,4","48,8","47,0","45,0","42,0","40,5"
2,OECD-Average,25-29,"107,9","105,0","103,1","103,5","103,4","102,3","102,6","103,4",...,"99,4","96,5","96,9","96,1","94,9","92,8","90,7","88,4","85,5","86,9"
3,OECD-Average,30-34,"88,1","87,2","88,6","91,1","93,4","94,6","97,6","100,0",...,"103,6","101,9","104,0","104,9","105,3","104,1","103,3","101,7","99,6","103,6"
4,OECD-Average,35-39,"38,1","38,6","39,5","40,6","42,2","43,5","45,6","47,7",...,"53,1","53,4","55,1","56,3","57,1","57,0","57,4","57,5","55,9","58,7"


In [949]:
# --- Ensure "country" and "age_group" are strings
df_19_3["country"] = df_19_3["country"].astype(str).str.strip().str.title()
df_19_3["age_group"] = df_19_3["age_group"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_19_3.columns if c not in ["country", "age_group"]]
# --- Robust cleaning -> convert to float ---
df_19_3[num_cols] = (
    df_19_3[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_19_3[num_cols] = df_19_3[num_cols].round(2)

In [950]:
df_19_3.drop_duplicates(inplace=True)
df_19_3.dropna(inplace=True)

In [951]:
#Check again
df_info = pd.DataFrame({
    'dtype': df_19_3.dtypes,
    'null_count': df_19_3.isnull().sum(),
    'unique_count': df_19_3.nunique()
})
print(df_info)

             dtype  null_count  unique_count
country     object           0            41
age_group   object           0             7
2000       float64           0           225
2001       float64           0           237
2002       float64           0           232
2003       float64           0           229
2004       float64           0           233
2005       float64           0           229
2006       float64           0           229
2007       float64           0           230
2008       float64           0           238
2009       float64           0           238
2010       float64           0           230
2011       float64           0           227
2012       float64           0           231
2013       float64           0           225
2014       float64           0           226
2015       float64           0           225
2016       float64           0           237
2017       float64           0           227
2018       float64           0           233
2019      

In [952]:
df_19_3.sample(10)

,country,age_group,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
193,New Zealand,35-39,52.5,53.7,54.7,58.7,60.2,62.3,63.7,70.3,...,69.5,69.8,66.4,70.8,67.8,65.3,63.70,64.80,60.7,62.6
76,Estonia,45-49,0.2,0.2,0.1,0.3,0.2,0.2,0.2,0.3,...,0.5,0.4,0.6,0.6,0.5,0.9,0.80,1.10,1.1,1.2
155,Latvia,20-24,81.7,78.5,76.6,78.5,74.8,74.1,76.8,77.2,...,58.7,60.2,63.5,64.4,64.3,62.6,56.00,54.10,53.8,46.6
239,Sweden,20-24,47.3,46.7,47.7,47.1,46.9,46.7,47.6,49.6,...,47.3,45.7,44.5,43.6,43.1,41.0,40.40,38.20,34.9,32.4
202,Norway,45-49,0.2,0.3,0.2,0.3,0.3,0.4,0.4,0.4,...,0.6,0.5,0.7,0.8,0.7,0.7,0.80,0.90,0.7,0.8
159,Latvia,40-44,3.8,4.2,4.3,4.7,5.1,4.9,5.7,6.4,...,7.6,9.4,10.0,11.3,12.2,13.5,12.80,13.60,13.2,13.5
278,Bulgaria,40-44,1.9,1.8,1.7,1.8,2.2,2.2,2.3,2.9,...,4.4,4.7,5.4,5.9,6.6,7.1,7.20,7.70,7.8,7.8
1,Oecd-Average,20-24,71.7,69.3,66.8,65.5,64.7,63.2,62.9,63.0,...,56.4,53.8,53.3,51.9,50.4,48.8,47.00,45.00,42.0,40.5
131,Israel,40-44,21.5,22.2,22.2,23.5,23.8,23.6,24.1,24.5,...,29.5,29.1,29.9,31.5,31.5,31.2,31.44,31.37,29.7,30.9
205,Poland,25-29,94.6,92.3,88.8,88.1,89.1,90.3,91.1,91.7,...,92.0,88.3,90.2,89.3,93.6,100.2,98.80,97.60,95.8,91.9


In [953]:
#df_19_3.to_csv('../data/Cleaned/cleaned_fertility_per_1000_from_2000_oecd.csv',index=False)

In [954]:
#df_19_3.to_sql('fertility_per_1000_from_2000_oecd',engine, if_exists='replace', index=False)

In [955]:
df_20= pd.read_csv('../data/Raw/OECD/SF_2_4_Share_births_outside_marriage_1960.csv')
#(%)share_of_births_outside_of_marriage
df_20

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Austria,"13,0","12,6","12,0","11,6","11,3","11,2","11,4","11,5","12,0",...,"40,4","41,5","41,4","41,7","42,1","42,2","42,0","41,3","40,6","41,2"
1,Belgium,"2,1","2,0","2,1","2,2","2,3","2,4","2,5","2,5","2,7",...,"47,0","47,7","49,5","49,4","48,0","49,0","52,8","52,4",..,..
2,Czech Republic,"4,9","4,6","4,5","4,7","4,8","5,0","5,3","5,3","5,4",...,"41,8","43,4","45,0","46,7","47,8","48,6","49,0","48,5","48,2","48,5"
3,Denmark,"7,8","8,0","8,3","8,9","9,3","9,5","10,2","11,1","11,1",...,"49,0","50,6","51,5","52,5","53,8","54,0","54,2","54,2","54,1","54,2"
4,Finland,"4,0","4,1","4,0","4,2","4,4","4,6","4,8","5,1","5,3",...,"40,9","41,5","42,1","42,8","44,3","44,9","44,8","44,6","45,4","46,1"
5,Germany,"7,6","7,1","6,6","6,1","5,9","5,8","5,7","5,8","6,1",...,"33,9","34,5","34,8","35,0","35,0","35,5","34,7","33,9","33,3","33,1"
6,Greece,"1,2","1,2","1,2","1,2","1,1","1,1","1,0","1,0","1,1",...,"7,4","7,6","7,0","8,2","8,8","9,4","10,3","11,1","12,4","13,8"
7,Hungary,"5,5","5,5","5,4","5,3","5,2","5,2","5,1","5,0","5,0",...,"42,3","44,5","45,6","47,3","47,9","46,7","44,7","43,9","38,7","30,4"
8,Iceland,"25,3","25,3","24,5","25,1","26,7","26,9","28,4","30,0","30,5",...,"65,0","66,9",..,..,..,"69,6","71,2","70,5","69,4",..
9,Ireland,"1,6","1,6","1,8","1,8","2,0","2,2","2,3","2,5","2,6",...,"33,9","35,1","35,3","36,3","36,6","36,7","37,6","37,9","38,4",..


In [956]:
df_info = pd.DataFrame({
    'dtype': df_20.dtypes,
    'null_count': df_20.isnull().sum(),
    'unique_count': df_20.nunique()
})
print(df_info)

          dtype  null_count  unique_count
Country  object           0            26
1960     object           0            26
1961     object           0            24
1962     object           0            24
1963     object           0            24
...         ...         ...           ...
2016     object           0            24
2017     object           0            26
2018     object           0            25
2019     object           0            25
2020     object           0            24

[62 rows x 3 columns]


In [957]:
df_20.columns = df_20.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [958]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_20.columns if c != "country"]

df_20[num_cols] = (
    df_20[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [959]:
df_20.drop_duplicates(inplace=True)
df_20.dropna(inplace=True)

df_info = pd.DataFrame({
    'dtype': df_20.dtypes,
    'null_count': df_20.isnull().sum(),
    'unique_count': df_20.nunique()
})
print(df_info)

           dtype  null_count  unique_count
country   object           0            22
1960     float64           0            22
1961     float64           0            20
1962     float64           0            21
1963     float64           0            21
...          ...         ...           ...
2016     float64           0            20
2017     float64           0            22
2018     float64           0            21
2019     float64           0            22
2020     float64           0            22

[62 rows x 3 columns]


In [960]:
df_20.sample(10)

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
7,Hungary,5.5,5.5,5.4,5.3,5.2,5.2,5.1,5.0,5.0,...,42.3,44.5,45.6,47.3,47.9,46.7,44.7,43.9,38.7,30.4
6,Greece,1.2,1.2,1.2,1.2,1.1,1.1,1.0,1.0,1.1,...,7.4,7.6,7.0,8.2,8.8,9.4,10.3,11.1,12.4,13.8
20,Sweden,11.3,11.7,12.4,12.6,13.1,13.8,14.5,15.1,16.0,...,54.3,54.5,54.4,54.6,54.7,54.9,54.5,54.5,54.5,55.2
4,Finland,4.0,4.1,4.0,4.2,4.4,4.6,4.8,5.1,5.3,...,40.9,41.5,42.1,42.8,44.3,44.9,44.8,44.6,45.4,46.1
17,Slovak Republic,4.7,4.4,4.6,4.7,5.0,5.3,5.3,5.7,5.9,...,34.0,35.4,37.0,38.9,39.2,40.2,40.1,40.0,40.1,40.7
3,Denmark,7.8,8.0,8.3,8.9,9.3,9.5,10.2,11.1,11.1,...,49.0,50.6,51.5,52.5,53.8,54.0,54.2,54.2,54.1,54.2
25,Croatia,7.4,7.1,6.5,6.9,6.8,6.0,5.7,5.4,5.5,...,14.0,15.4,16.1,17.4,18.1,18.9,19.9,20.7,21.5,22.8
14,New Zealand,4.6,5.1,8.0,8.8,9.9,10.9,11.6,12.7,13.0,...,47.6,47.7,47.3,46.7,46.7,45.9,46.9,48.2,47.6,48.3
13,Netherlands,1.4,1.4,1.5,1.6,1.7,1.8,2.0,2.1,2.0,...,45.3,46.6,47.4,48.7,49.8,50.4,51.0,51.9,52.4,53.5
12,Luxembourg,3.2,3.4,3.1,3.1,3.2,3.7,3.2,3.5,3.2,...,34.1,37.1,37.8,39.1,38.8,40.7,40.8,39.5,40.4,41.6


In [961]:
#df_20.to_csv('../data/Cleaned/cleaned_share_of_births_outside_of_marriage_oecd.csv', index=False)

In [962]:
#df_20.to_sql('share_of_births_outside_of_marriage_oecd',engine, if_exists='replace', index=False)

In [963]:
df_21_1= pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rate_mean_age_first_marriage_S1.csv')
#mean_age_first_marriage
df_21_1

,Country,Gender,1990,1991,1992,1993,1994,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Australia,Male,"26,5","26,7","26,9","27,0","27,2","27,3","27,6","27,8",...,"29,7","29,8","29,9","30,0","30,1","30,3","30,4","30,7","30,7","30,6"
1,Australia,Female,"24,3","24,5","24,7","24,8","25,1","25,3","25,7","25,9",...,"28,0","28,1","28,3","28,4","28,5","28,7","28,8","29,2","29,3","29,2"
2,Czechia,Male,"24,3","24,3","24,5","24,7","25,1","25,5","25,9","26,5",...,"31,0","31,2","31,3","31,4","31,6","31,7","31,8","31,9","32,0","32,4"
3,Czechia,Female,"21,6","21,6","21,9","22,1","22,4","22,8","23,1","23,6",...,"28,1","28,3","28,5","28,7","28,8","29,0","29,1","29,2","29,4","29,7"
4,Denmark,Male,"30,5","30,6","31,0","31,4","31,8","31,9","32,5","32,2",...,"33,8","34,3","34,4","34,4","34,3","34,7","34,8","34,9","35,1","35,3"
5,Denmark,Female,"27,8","28,0","28,3","28,8","29,2","29,2","29,9","30,1",...,"31,4","31,8","31,9","31,9","31,9","32,2","32,4","32,5","32,8","33,0"
6,Greece,Male,"29,0","29,3","29,6","29,7","29,9","30,1","30,2","30,6",...,"32,7","32,8","32,9","33,0","33,2","33,2","33,3","33,4","33,7","33,8"
7,Greece,Female,"24,9","25,2","25,5","25,5","25,8","26,0","26,3","26,6",...,"29,4","29,5","29,7","29,9","30,1","30,1","30,3","30,3","30,7","30,7"
8,Japan,Male,"28,4","28,4","28,4","28,4","28,5","28,5","28,5","28,5",...,"30,7","30,8","30,9","31,1","31,1","31,1","31,1","31,1","31,2","31,0"
9,Japan,Female,"25,9","25,9","26,0","26,1","26,2","26,3","26,4","26,6",...,"29,0","29,2","29,3","29,4","29,4","29,4","29,4","29,4","29,6","29,4"


In [964]:
df_info = pd.DataFrame({
    'datatypes': df_21_1.dtypes,
    'null_count': df_21_1.isnull().sum(),
    'unique_count': df_21_1.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
Country    object           0            10
Gender     object           0             2
1990       object           0            17
1991       object           0            18
1992       object           0            18
1993       object           0            19
1994       object           0            16
1995       object           0            18
1996       object           0            19
1997       object           0            17
1998       object           0            14
1999       object           0            19
2000       object           0            18
2001       object           0            18
2002       object           0            19
2003       object           0            19
2004       object           0            16
2005       object           0            18
2006       object           0            18
2007       object           0            19
2008       object           0            18
2009       object           0   

In [965]:
df_21_1.columns = df_21_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [966]:
# --- Ensure "country" and "gender" are strings
df_21_1["country"] = df_21_1["country"].astype(str).str.strip().str.title()
df_21_1["gender"] = df_21_1["gender"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_21_1.columns if c not in ["country", "gender"]]
# --- Robust cleaning -> convert to float ---
df_21_1[num_cols] = (
    df_21_1[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_21_1[num_cols] = df_21_1[num_cols].round(2)

In [967]:
df_21_1.drop_duplicates(inplace=True)
df_21_1.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_21_1.dtypes,
    'null_count': df_21_1.isnull().sum(),
    'unique_count': df_21_1.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
country    object           0             9
gender     object           0             2
1990      float64           0            15
1991      float64           0            16
1992      float64           0            16
1993      float64           0            17
1994      float64           0            15
1995      float64           0            16
1996      float64           0            17
1997      float64           0            15
1998      float64           0            13
1999      float64           0            17
2000      float64           0            16
2001      float64           0            16
2002      float64           0            17
2003      float64           0            17
2004      float64           0            15
2005      float64           0            17
2006      float64           0            17
2007      float64           0            17
2008      float64           0            16
2009      float64           0   

In [968]:
#df_21_1.to_csv('../data/Cleaned/cleaned_mean_age_first_marriage_oecd.csv',index=False)

In [969]:
#df_21_1.to_sql('mean_age_first_marriage_oecd', engine, if_exists='replace', index= False)

In [970]:
df_21_2 = pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rates_S2.csv')
#divorce_rates_per_1000_oecd
df_21_2

,Country,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Austria,"1,4","1,3","1,3","1,3","1,4","1,4","1,5","1,5","1,6",...,"1,9","1,9","1,9","1,8","1,8","1,8","1,8","1,7","1,6","1,5"
1,Belgium,"0,7","0,7","0,8","0,9","1,0","1,1","1,3","1,3","1,4",...,"2,2","2,2","2,2","2,1","2,0","2,0","2,0","1,8","1,9","1,7"
2,Czechia,"2,2","2,4","2,3","2,5","2,5","2,6","2,5","2,5","2,6",...,"2,7","2,5","2,5","2,4","2,4","2,3","2,3","2,0","2,0","1,9"
3,Denmark,"1,9","2,7","2,6","2,5","2,6","2,6","2,6","2,6","2,6",...,"3,4","3,4","2,9","3,0","2,6","2,6","1,8","2,7","2,2","2,1"
4,Estonia,"3,2","3,2","3,3","3,2","3,3","3,4","3,6","3,9","3,8",...,"2,5","2,4","2,6","2,5","2,5","2,4","2,1","1,9",NaN,"1,9"
5,Finland,"1,3","1,6","1,8","1,9","2,1","2,0","2,1","2,1","2,2",...,"2,5","2,5","2,5","2,5","2,4","2,4","2,4","2,4","2,2","2,0"
6,Germany,"1,3","1,4","1,5","1,6","1,8","1,9","2,0","1,5","1,0",...,"2,1","2,1","2,0","2,0","1,9","1,8","1,8","1,7","1,7","1,6"
7,Greece,"0,4","0,4","0,4","0,5","0,4","0,4","0,4","0,5","0,5",...,"1,5","1,3","1,4","1,0","1,8",NaN,NaN,NaN,NaN,NaN
8,Hungary,"2,2","2,3","2,3","2,4","2,3","2,5","2,6","2,6","2,7",...,"2,0","2,0","2,1","2,0","1,9","1,7","1,8","1,5","1,9","1,8"
9,Italy,..,"0,3","0,6","0,3","0,3","0,2","0,2","0,2","0,2",...,"0,9","0,9","1,4","1,6","1,5","1,5","1,4","1,1","1,4","1,4"


In [971]:
df_info = pd.DataFrame({
    'datatypes': df_21_2.dtypes,
    'null_count': df_21_2.isnull().sum(),
    'unique_count': df_21_2.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
Country    object           0            28
1970       object           0            18
1971       object           0            19
1972       object           0            19
1973       object           0            18
1974       object           0            18
1975       object           0            19
1976       object           0            18
1977       object           0            18
1978       object           0            18
1979       object           0            15
1980       object           0            18
1981       object           0            20
1982       object           0            22
1983       object           0            24
1984       object           0            20
1985       object           0            19
1986       object           0            20
1987       object           0            20
1988       object           0            20
1989       object           0            19
1990       object           0   

In [972]:
df_21_2.columns = df_21_2.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [973]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_21_2.columns if c != "country"]

df_21_2[num_cols] = (
    df_21_2[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [974]:
df_21_2.drop_duplicates(inplace=True)
df_21_2.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_21_2.dtypes,
    'null_count': df_21_2.isnull().sum(),
    'unique_count': df_21_2.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
country    object           0            23
1970      float64           0            15
1971      float64           0            17
1972      float64           0            15
1973      float64           0            14
1974      float64           0            15
1975      float64           0            16
1976      float64           0            14
1977      float64           0            13
1978      float64           0            15
1979      float64           0            12
1980      float64           0            14
1981      float64           0            17
1982      float64           0            17
1983      float64           0            19
1984      float64           0            16
1985      float64           0            15
1986      float64           0            16
1987      float64           0            16
1988      float64           0            15
1989      float64           0            15
1990      float64           0   

In [975]:
df_21_2.head(8)

,country,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Austria,1.4,1.3,1.3,1.3,1.4,1.4,1.5,1.5,1.6,...,1.9,1.9,1.9,1.8,1.8,1.8,1.8,1.7,1.60,1.50
1,Belgium,0.7,0.7,0.8,0.9,1.0,1.1,1.3,1.3,1.4,...,2.2,2.2,2.2,2.1,2.0,2.0,2.0,1.8,1.90,1.70
2,Czechia,2.2,2.4,2.3,2.5,2.5,2.6,2.5,2.5,2.6,...,2.7,2.5,2.5,2.4,2.4,2.3,2.3,2.0,2.00,1.90
3,Denmark,1.9,2.7,2.6,2.5,2.6,2.6,2.6,2.6,2.6,...,3.4,3.4,2.9,3.0,2.6,2.6,1.8,2.7,2.20,2.10
5,Finland,1.3,1.6,1.8,1.9,2.1,2.0,2.1,2.1,2.2,...,2.5,2.5,2.5,2.5,2.4,2.4,2.4,2.4,2.20,2.00
6,Germany,1.3,1.4,1.5,1.6,1.8,1.9,2.0,1.5,1.0,...,2.1,2.1,2.0,2.0,1.9,1.8,1.8,1.7,1.70,1.60
8,Hungary,2.2,2.3,2.3,2.4,2.3,2.5,2.6,2.6,2.7,...,2.0,2.0,2.1,2.0,1.9,1.7,1.8,1.5,1.90,1.80
10,Japan,0.9,1.0,1.0,1.0,1.0,1.1,1.1,1.1,1.2,...,1.8,1.8,1.8,1.7,1.7,1.7,1.7,1.6,1.47,1.52


In [976]:
#df_21_2.to_csv('../data/Cleaned/cleaned_divorce_rates_per_1000_oecd.csv', index=False)

In [977]:
#df_21_2.to_sql('divorce_rates_per_1000_oecd',engine, if_exists= 'replace' , index=False)

In [978]:
df_21_3= pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rates_prev_marital_status_S3.csv')
#share_of_previous_marital_status
df_21_3

,Country,Previous marital status,2000,2001,2002,2003,2004,2005,2006,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Australia,Single never married,"75,9","76,1","75,5","75,6","76,2","76,9","77,3","78,2",...,"79,6","79,7","80,0","80,5","80,5","80,1","80,3","80,1","80,3","80,7"
1,Australia,Divorced,"22,0","21,8","22,4","22,3","21,8","21,3","20,9","20,2",...,"19,0","18,8","18,6","18,2","18,1","18,5","18,3","18,5","18,3","18,0"
2,Australia,Widowed,"2,1","2,1","2,1","2,1","1,9","1,8","1,8","1,7",...,"1,5","1,5","1,4","1,3","1,4","1,4","1,4","1,4","1,3","1,3"
3,Austria,Single never married,"76,6","74,7","74,1","73,7","72,9","73,1","73,9","74,8",...,"75,5","75,7","76,7","77,1","77,5","77,7","78,1","78,1","78,2","78,0"
4,Austria,Divorced,"22,2","24,2","24,7","25,2","25,9","25,7","24,9","24,2",...,"23,5","23,4","22,3","22,0","21,5","21,5","20,9","21,0","21,0","21,6"
5,Austria,Widowed,"1,2","1,1","1,2","1,1","1,2","1,2","1,1","1,0",...,"1,0","0,9","1,0","0,9","1,0","0,8","0,9","0,9","0,8","0,4"
6,Czechia,Single never married,"74,9","74,5","74,3","74,0","73,9","74,2","74,5","72,6",...,"74,0","74,0","75,2","75,6","76,6","76,7","76,4","76,4","76,1","75,9"
7,Czechia,Divorced,"23,7","24,2","24,4","24,7","24,7","24,5","24,4","26,1",...,"24,9","24,9","23,8","23,4","22,4","22,3","22,6","22,6","22,9","23,0"
8,Czechia,Widowed,"1,4","1,3","1,3","1,3","1,4","1,2","1,1","1,3",...,"1,2","1,1","1,0","1,0","1,0","1,0","1,0","1,0","1,0","1,1"
9,Denmark,Single never married,"75,9","76,0","76,2","76,4","76,0","75,6","75,6","76,3",...,"77,2","76,0","75,0","76,2","76,1","76,9","76,4","77,1","77,6","78,3"


In [979]:
df_info = pd.DataFrame({
    'datatypes': df_21_3.dtypes,
    'null_count': df_21_3.isnull().sum(),
    'unique_count': df_21_3.nunique()
})
print(df_info)

                        datatypes  null_count  unique_count
Country                    object           0            20
Previous marital status    object           0             3
2000                       object           0            47
2001                       object           0            51
2002                       object           0            56
2003                       object           0            50
2004                       object           0            50
2005                       object           0            52
2006                       object           0            49
2008                       object           0            47
2009                       object           0            50
2010                       object           0            49
2011                       object           0            49
2012                       object           0            53
2013                       object           0            49
2014                       object       

In [980]:
df_21_3.columns = df_21_3.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_21_3.head()

,country,previous_marital_status,2000,2001,2002,2003,2004,2005,2006,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Australia,Single never married,"75,9","76,1","75,5","75,6","76,2","76,9","77,3","78,2",...,"79,6","79,7","80,0","80,5","80,5","80,1","80,3","80,1","80,3","80,7"
1,Australia,Divorced,"22,0","21,8","22,4","22,3","21,8","21,3","20,9","20,2",...,"19,0","18,8","18,6","18,2","18,1","18,5","18,3","18,5","18,3","18,0"
2,Australia,Widowed,"2,1","2,1","2,1","2,1","1,9","1,8","1,8","1,7",...,"1,5","1,5","1,4","1,3","1,4","1,4","1,4","1,4","1,3","1,3"
3,Austria,Single never married,"76,6","74,7","74,1","73,7","72,9","73,1","73,9","74,8",...,"75,5","75,7","76,7","77,1","77,5","77,7","78,1","78,1","78,2","78,0"
4,Austria,Divorced,"22,2","24,2","24,7","25,2","25,9","25,7","24,9","24,2",...,"23,5","23,4","22,3","22,0","21,5","21,5","20,9","21,0","21,0","21,6"


In [981]:
# --- Ensure "country" and "previous_marital_status" are strings
df_21_3["country"] = df_21_3["country"].astype(str).str.strip().str.title()
df_21_3["previous_marital_status"] = df_21_3["previous_marital_status"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_21_3.columns if c not in ["country", "previous_marital_status"]]
# --- Robust cleaning -> convert to float ---
df_21_3[num_cols] = (
    df_21_3[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_21_3[num_cols] = df_21_3[num_cols].round(2)

In [982]:
df_21_3.drop_duplicates(inplace=True)
df_21_3.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_21_3.dtypes,
    'null_count': df_21_3.isnull().sum(),
    'unique_count': df_21_3.nunique()
})
print(df_info)

                        datatypes  null_count  unique_count
country                    object           0            20
previous_marital_status    object           0             3
2000                      float64           0            47
2001                      float64           0            51
2002                      float64           0            56
2003                      float64           0            50
2004                      float64           0            50
2005                      float64           0            52
2006                      float64           0            49
2008                      float64           0            47
2009                      float64           0            50
2010                      float64           0            49
2011                      float64           0            49
2012                      float64           0            53
2013                      float64           0            49
2014                      float64       

In [ ]:
#df_21_3.to_csv('../data/Cleaned/cleaned_share_of_previous_marital_status_oecd.csv', index=False)

In [984]:
#df_21_3.to_sql('share_of_previous_marital_status_oecd', engine, if_exists= 'replace', index =  False)

In [1002]:
df_22_1 = pd.read_csv('../data/Raw/OECD/SF3_3_A_in_private_households_by_partnership_status_S1.csv')
df_22_1

,Country,20+_All_Total_Living_with_a_partner(%),20+_All_Married or in a civil or registered partnership_living_with_a_partner(%),20+_All_Cohabiting_living_with_a_partner(%),20+_All_Not living with a partner(%),20/34_Total_living_with_a_partner(%),20/34_Married or in a civil or registered partnership_living_with_a_partner(%),20/34_Cohabiting_living_with_a_partner(%),Not living with a partner_Total(%),Living with at least one parent(%)
0,Australia (c),"63,79","53,59","10,20","36,21","47,06","29,41","17,65","52,94",..
1,Austria,"58,80","49,10","9,70","41,20","39,11","22,15","16,97","60,89","33,82"
2,Belgium,"62,15","53,51","8,64","37,85","45,28","29,33","15,94","54,72","31,34"
3,Canada (d),"66,89","54,46","12,43","33,11","55,34","33,55","21,79","44,66",..
4,Czech Republic,"51,17","45,39","5,79","48,83","30,78","21,32","9,46","69,22","36,20"
5,Denmark,"64,15","50,02","14,12","35,85","50,54","21,86","28,68","49,46","10,67"
6,Estonia,"53,93","37,30","16,64","46,07","45,31","17,81","27,50","54,69","26,46"
7,France,"64,14","49,41","14,72","35,86","50,42","21,89","28,53","49,58","22,08"
8,Germany,"62,61","53,91","8,69","37,39","39,53","22,15","17,39","59,74","27,54"
9,Greece,"60,23","58,52","1,71","39,77","33,13","29,24","3,90","66,87","45,43"


In [995]:
df_info = pd.DataFrame({
    'datatypes': df_22_1.dtypes,
    'null_count': df_22_1.isnull().sum(),
    'unique_count': df_22_1.nunique()
})
print(df_info)

                                                   datatypes  null_count  \
Country                                               object           0   
20+_All_Total_Living_with_a_partner(%)                object           0   
20+_All_Married or in a civil or registered par...    object           0   
20+_All_Cohabiting_living_with_a_partner(%)           object           0   
20+_All_Not living with a partner(%)                  object           0   
20/34_Total_living_with_a_partner(%)                  object           0   
20/34_Married or in a civil or registered partn...    object           0   
20/34_Cohabiting_living_with_a_partner(%)             object           0   
Not living with a partner_Total(%)                    object           0   
Living with at least one parent(%)                    object           0   

                                                    unique_count  
Country                                                       37  
20+_All_Total_Living_with_a_p

In [1003]:
df_22_1.columns = df_22_1.columns.str.lower() \
                .str.replace(' ', '_') \


df_22_1.head()

,country,20+_all_total_living_with_a_partner(%),20+_all_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),20+_all_cohabiting_living_with_a_partner(%),20+_all_not_living_with_a_partner(%),20/34_total_living_with_a_partner(%),20/34_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),20/34_cohabiting_living_with_a_partner(%),not_living_with_a_partner_total(%),living_with_at_least_one_parent(%)
0,Australia (c),"63,79","53,59","10,20","36,21","47,06","29,41","17,65","52,94",..
1,Austria,"58,80","49,10","9,70","41,20","39,11","22,15","16,97","60,89","33,82"
2,Belgium,"62,15","53,51","8,64","37,85","45,28","29,33","15,94","54,72","31,34"
3,Canada (d),"66,89","54,46","12,43","33,11","55,34","33,55","21,79","44,66",..
4,Czech Republic,"51,17","45,39","5,79","48,83","30,78","21,32","9,46","69,22","36,20"


In [ ]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_22_1.columns if c != "country"]

df_22_1[num_cols] = (
    df_22_1[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")

SyntaxError: incomplete input (2079695211.py, line 10)

In [ ]:
df_22_1

In [986]:
df_22_2 = pd.read_csv('../data/Raw/OECD/SF3_3_B_ by level of educational attainment_S2.csv')
df_22_2

,Country,Low_Education_Total_living_with_a_partner(%),Low_educationMarried or in a civil or registered partnership_living_with_a_partner(%),Low_education_Cohabiting_living_with_a_partner(%),Not living with a partner_Low_education(%),Medium education_Total_Living with a partner(%),Medium education_Married or in a civil or registered partnership_Living with a partner(%),Medium education_Cohabiting_Living with a partner(%),Not living with a partner_Medium education(%),High education_Total_Living with a partner(%),High education_Married or in a civil or registered partnership_Living with a partner(%),High education_Cohabiting_Living with a partner(%),Not living with a partner_High education(%)
0,Austria,"56,81","50,49","6,32","43,19","59,27","48,73","10,54",NaN,"60,03","48,38","11,65","39,97"
1,Belgium,"62,28","56,11","6,17","37,72","60,79","49,80","10,99",NaN,"67,09","56,58","10,51","32,91"
2,Czech Republic,"40,81","36,55","4,26","59,19","53,99","47,87","6,12","46,01","57,29","50,26","7,03","42,71"
3,Estonia,"42,17","26,39","15,78","57,83","54,41","36,61","17,79","45,59","60,14","44,45","15,69","39,86"
4,France,"61,12","51,93","9,18","38,88","65,68","49,17","16,51","34,32","65,58","46,60","18,98","34,42"
5,Germany,"54,46","48,79","5,67","45,54","62,38","53,13","9,25","37,62","68,89","59,16","9,74","31,11"
6,Greece,"63,81","62,88","0,93","36,19","57,00","54,88","2,12","43,00","58,33","55,70","2,63","41,67"
7,Hungary,"50,33","40,38","9,95","49,67","57,94","46,78","11,15","42,06","59,56","51,02","8,55","40,44"
8,Iceland,"51,86","41,02","10,84","48,14","58,31","46,57","11,74","41,69","69,72","54,53","15,19","30,28"
9,Latvia,"36,27","25,92","10,35","63,73","49,32","39,54","9,78","50,68","52,91","45,39","7,52","47,09"


In [987]:
df_6666 = pd.read_csv('../data/Raw/OECD/OECD_df_famliy_selected.csv')
df_6666

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,COU,Country,SEX,Sex,IND,Indicator,...,OBS_VALUE,Observation Value,OBS_STATUS,Observation Status,UNIT_MEASURE,Unit of Measures,UNIT_MULT,Multiplier,BASE_PER,Base reference period
0,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,LVA,Latvia,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,39.5,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
1,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,GRC,Greece,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,11.1,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
2,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,CHL,Chile,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,74.8,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
3,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,NLD,Netherlands,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,51.9,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
4,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,LTU,Lithuania,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,26.4,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,COL,Colombia,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.4,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
501,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.5,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
502,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.6,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
503,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.7,NaN,A,NaN,YR,Years,0,Units,NaN,NaN


In [988]:
df_888= pd.read_csv('../data/Raw/OECD/Households-by-type,-presence-of-children-and-country,-2015-2024.csv')
df_888

,Category,Single adult with children,Single adult without children,Couple with children,Couple without children,Other type of household with children,Other type of household without children
0,2015,6147.3,64181.3,31679.8,46641.6,11698.9,30771.6
1,2016,6148.5,63891.1,31907.3,47308.2,11766.3,30559.5
2,2017,6108.5,65353.9,32091.5,47426.1,11530.2,30297.5
3,2018,6163.6,66165.5,31720.2,48194.8,11342.5,30224.0
4,2019,6246.4,67417.9,31710.1,48503.6,11285.7,30134.8
5,2020,6136.4,67412.9,31622.2,48831.2,11212.9,30445.2
6,2021,5691.9,70200.4,30558.3,47447.4,11611.8,30700.7
7,2022,5984.9,72134.3,30469.3,47995.5,11513.6,30412.1
8,2023,5924.8,73396.2,30313.0,48477.5,11443.5,30608.8
9,2024,6077.7,75049.7,30286.5,49058.4,11311.9,30487.3


In [989]:
df_999 = pd.read_csv('../data/Raw/OECD/Households-with-children-by-number-of-children,-2024.csv')
df_999

,Category,1 child,2 children,3 children or more
0,European Union,11.7,8.9,3.0
1,NaN,NaN,NaN,NaN
2,Slovakia,17.1,14.5,4.0
3,Ireland,12.4,12.2,6.4
4,Cyprus,13.9,11.7,3.1
5,Czechia,13.9,11.6,2.6
6,Romania,14.3,9.2,4.0
7,Luxembourg,12.5,12.1,2.4
8,Belgium,11.8,10.2,4.1
9,Croatia,12.0,10.1,3.8
